In [1]:
import numpy as np
import pandas
import string
import urllib
from bs4 import BeautifulSoup as bs
import datetime
import dateutil
from pprint import pprint
import os
import re
from datetime import datetime

pandas.set_option('max_colwidth',-1)
print('ok')

ok


In [2]:
# Read dataset
os.chdir('/home/idies/workspace/raddick_folksonomy/data/')
os.getcwd()
inmusic_df = pandas.read_csv('cleaned_audio_data.csv')
# Use music_df for processing; move rows to output when scored
music_df = inmusic_df
# IDs are unique, we can use them as pandices IWW - but now let's use rownumber #music_df.groupby('ID').size().sort_values(ascending=False)
print('Datafile read')

# rename columns
music_df.index.name = 'rownumber'
music_df = music_df.rename(columns={'ID': 'id', 'Source': 'source', 'Sample File Name': 'filename'})
print('Columns renamed')

### SUMMARY STATISTICS FOR FILENAME DATASET
print(len(music_df))
# 2,973 filenames

print('\n')
print(
    music_df
    .groupby('source')
    .size()
    .sort_values(ascending=False)
)
# Sources:
#DimeaDozen      1804
#Etree            564
#LosslessLegs     275
#TradersDen       330

print('\n')
print(
    music_df['filename'].
    groupby(music_df['filename'].
            apply(lambda x: x.lower().split('.')[-1])).
    size().
    sort_values(ascending=False)
)
# File extensions:
#flac    2798
#shn       92
#mp2       48
#mp3       31
#wav        1
#cue        1
#aud        1
#FLAC       1
print('ok')

Datafile read
Columns renamed
2973


source
DimeaDozen      1804
Etree           564 
TradersDen      330 
LosslessLegs    275 
dtype: int64


filename
flac    2799
shn     92  
mp2     48  
mp3     31  
wav     1   
cue     1   
aud     1   
Name: filename, dtype: int64
ok


In [ ]:
'''
STEPS IN FLOWCHART:
A. Does filename start with the artist name? (False -> 0)
B. Does the date come next? (False -> 0)
C. Does an artist name come next? (False -> 0)

D. Does the artist have an official abbrev? (True -> E, False -> G)

E. Is (abbrev used correctly) OR (date correctly formatted)? (True -> F , False -> 1)
F. Is (abbrev used correctly) AND (date correctly formatted)? (True -> H , False -> 2)

G. Is the date correctly formatted? (True -> H, False -> 1)

H. Does the date immediately follow artist name, no space between? (False -> 3)

J. Do they include the mics parameter? (True -> K, False -> L)
K. Do they use the mics parameter correctly? (True -> L, False -> 4)

L. Is taper used correctly? (False -> 4)
M. Is sbesource used correctly? (False -> 4)
N. Is filetype used correctly? (False -> 4)
P. Does filetype accurately describe the type of file? (True -> 5, False -> 4)
'''
# Researching the .cue file that snuck into the dataset...
#adf = inmusic_df
#adf['Sample File Name'] = adf['Sample File Name'].apply(lambda x: x.lower())
#adf[adf['Sample File Name'].apply(lambda x: '.cue' in x)]
print('ok')

In [3]:
# GET LIST OF OFFICIAL ABBREVIATIONS

print('Retrieving from etree')
pageurl = 'http://wiki.etree.org/index.php?page=BandAbbreviations'
page = urllib.request.urlopen(pageurl).read()
soup = bs(page, "html.parser")
tables = soup.find_all('table')

#for thistable in tables:
#    print(thistable)
#    print('---------')

bandtable = tables[1]
bands = bandtable.find_all('tr')
allbands = []

print('Thinking...')
for thisband in bands[1:]:
    theirdata = thisband.find_all('td')
    oneband = {}
    oneband['name'] = theirdata[0].text
    oneband['abbrev'] = theirdata[1].text
    try:
        oneband['otherabbrev'] = theirdata[2].text
    except:
        pass
    try:
        oneband['notes'] = theirdata[3].text
    except:
        pass
    allbands.append(oneband)
abbrev_df = pandas.DataFrame.from_records(allbands)
abbrev_df.index.name = 'rownumber'

column_order = ['name', 'abbrev', 'otherabbrev', 'notes']
abbrev_df = abbrev_df[column_order]
abbrev_df = abbrev_df.fillna('')

# Trim one space from beginning and end of band name
abbrev_df['name'] = abbrev_df['name'].apply(lambda x: str(x).strip())

# Trim away brackets from beginning and end of band name (for some bands)
abbrev_df['name'] = abbrev_df['name'] = np.where((abbrev_df['name'].apply(lambda x: (str(x)[0])) == '['), 
                             abbrev_df['name'].apply(lambda x: str(x)[1:]), 
                             abbrev_df['name']) 

abbrev_df['name'] = abbrev_df['name'] = np.where((abbrev_df['name'].apply(lambda x: (str(x)[len(str(x))-1])) == ']'),
                              abbrev_df['name'].apply(lambda x: str(x)[:-1]), 
                              abbrev_df['name'])

abbrev_df['abbrev'] = abbrev_df['abbrev'].apply(lambda x: str(x).lower().strip())
abbrev_df['otherabbrev'] = abbrev_df['otherabbrev'].apply(lambda x: str(x).lower().strip())

#abbrev_df['abbrev'].apply(lambda x: '|'+str(x)+'|')

abbrev_df.to_csv('abbreviations.csv')
os.listdir()
#print('ok')
#pprint(abbrev_df.values.tolist())
canonical_abbrevs = abbrev_df['abbrev'][abbrev_df['abbrev'] != ''].drop_duplicates().tolist()
#canonical_abbrevs

# Get variant abbreviations
variant_abbrevs = []
for x in abbrev_df['otherabbrev'].drop_duplicates():
    if (',' in x):
        for y in x.split(','):
            variant_abbrevs.append(y.strip())
    else:
        if (x.strip() != ''):
            variant_abbrevs.append(x.strip())

variant_abbrevs = sorted(variant_abbrevs)

# Fix wilco by moving it from variant to canonical
canonical_abbrevs.append('wilco')
variant_abbrevs.remove('wilco')

# Strip and lower
canonical_abbrevs = [x.lower().strip() for x in canonical_abbrevs]    
variant_abbrevs = [x.lower().strip() for x in variant_abbrevs]    

print('Parsed {0:,.0f} canonical abbreviations and {1:,.0f} variant abbreviations.'.format(len(canonical_abbrevs), len(variant_abbrevs)))

Retrieving from etree
Thinking...
Parsed 156 canonical abbreviations and 61 variant abbreviations.


In [4]:
debug = 1
markers = re.compile('[\.\-\_\ ]*')
endmarkers = re.compile('[\.\-\_\ ]$')
startswithbadness = '^[\s\_\-]*(dis|cd|track|artist|unknown)[\s\_\-]*'
startswithdigit = re.compile('^[\d]+')
dateformat = re.compile('[1-9]\d*[\s\/\-]*\d+[\s\/\-]*\d+')
daterightafterband = re.compile('[\d\s\_\-\.\ ]$')
bandtrimmer = re.compile('\s*[\s\_\-\.\ ]$')
scbands = re.compile('(calloway|earth|gregory|bley|smith|jimmy|zeppelin|redford|lizzy|beck|dust|peter gabriel|stones|peppers|sinatra)')
ends_with_dash_or_whatever = re.compile('\s*[\-\_\.]$')

os.chdir('/home/idies/workspace/raddick_folksonomy/data/')
os.getcwd()
inmusic_df = pandas.read_csv('cleaned_audio_data.csv')
# Use music_df for processing; move rows to output when scored
music_df = inmusic_df
# IDs are unique, we can use them as pandices IWW - but now let's use rownumber #music_df.groupby('ID').size().sort_values(ascending=False)

# rename columns
music_df.index.name = 'rownumber'
music_df = music_df.rename(columns={'ID': 'id', 'Source': 'source', 'Sample File Name': 'filename'})

print('Filenames in dataset: {0:,.0f}'.format(len(music_df)))

# First, strip filename and convert to lowercase.
music_df['filename'] = music_df['filename'].apply(lambda x: x.lower().strip())

# Next, get filetype (whatever is after the final period)
music_df['filetype'] = music_df['filename'].apply(lambda x: x.split('.')[-1].strip())
music_df['remainder'] = music_df['filename'].apply(lambda x: x[0:x.rfind('.')])


zeros_df = pandas.DataFrame(data=None, columns=music_df.columns)
#ones_df = pandas.DataFrame(data=None, columns=music_df.columns)
#twos_df = pandas.DataFrame(data=None, columns=music_df.columns)
#threes_df = pandas.DataFrame(data=None, columns=music_df.columns)
#fours_df = pandas.DataFrame(data=None, columns=music_df.columns)

zeros_df.index.name = music_df.index.name
#ones_df.index.name = music_df.index.name
#twos_df.index.name = music_df.index.name
#threes_df.index.name = music_df.index.name
#fours_df.index.name = music_df.index.name

# If there are no numbers in the remainder part of filename, score as zero and remove
print('\nStep 1. Filenames with no numbers in remainder part: score = 0')
zeros_df = zeros_df.append(music_df.loc[music_df['remainder'].apply(lambda x: re.search('\d', x) == None)])
music_df = music_df[~music_df.isin(zeros_df)].dropna()
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

# If starts with track or disc or cd and has a number immediately after, score as zero.
print('\nStep 2a. Filenames with track/cd/disc followed by number with optional whitespace: score = 0')
zeros_df = zeros_df.append(music_df.loc[music_df['remainder'].apply(lambda x: re.search(startswithbadness, x) != None)])
music_df = music_df[~music_df.isin(zeros_df)].dropna()
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

# If starts with track or disc or cd and has a number immediately after, score as zero.
print('\nStep 2b. Starts with a number but has no dates and is not a special case band: score = 0')
zeros_df = zeros_df.append(music_df.loc[music_df['filename'].apply(lambda x: 
               (re.search(startswithdigit, x) != None) & 
               (re.search(dateformat, x) == None) &
               (re.search(scbands, x) == None)                                                
              )])
music_df = music_df[~music_df.isin(zeros_df)].dropna()
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

# If starts with track or disc or cd and has a number immediately after, score as zero.
print('\nStep 2c. Remainder has only number: score = 0')
zeros_df = zeros_df.append(music_df.loc[music_df['remainder'].apply(lambda x: re.search('\D', x) == None)])
music_df = music_df[~music_df.isin(zeros_df)].dropna()
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

Filenames in dataset: 2,973

Step 1. Filenames with no numbers in remainder part: score = 0
Filenames remaining in dataset: 2,955

Step 2a. Filenames with track/cd/disc followed by number with optional whitespace: score = 0
Filenames remaining in dataset: 2,735

Step 2b. Starts with a number but has no dates and is not a special case band: score = 0
Filenames remaining in dataset: 1,805

Step 2c. Remainder has only number: score = 0
Filenames remaining in dataset: 1,803


In [5]:
# Parse up to first digit as band name
music_df['band'] = music_df['remainder'].apply(lambda x: x[0:re.search('\d', x).start()])

# Find whether the date immediately follows the band name (as the format demands)
music_df['date_right_after_band'] = False
music_df.loc[music_df['band'].apply(lambda x: re.search(daterightafterband, x) == None), 'date_right_after_band'] = True

# Strip and lower band names
music_df['band'] = music_df['band'].apply(lambda x: x.strip().lower())

# If the band name has a _ or - immediately after, strip that away
music_df.loc[music_df['band'].apply(lambda x: re.search(ends_with_dash_or_whatever, x) != None), 
             'band'] = music_df['band'][music_df['band'].apply(lambda x: re.search(ends_with_dash_or_whatever, x) != None)
                                       ].apply(lambda x: x[0:re.search(ends_with_dash_or_whatever, x).start()])

# Fix mistaken bands d, t, a, n, and tt
music_df.loc[
    (music_df['band'] == 'd') | (music_df['band'] == 't') | (music_df['band'] == 'a') | (music_df['band'] == 'n') | (music_df['band'] == 'tt'), 
    'date_right_after_band'] = False
music_df.loc[
    (music_df['band'] == 'd') | (music_df['band'] == 't') | (music_df['band'] == 'a') | (music_df['band'] == 'n') | (music_df['band'] == 'tt'), 
    'band'] = ''

# Fix U2
music_df.loc[music_df['band'] == 'u', 'band'] = 'u2'

# A few other bands to fix
music_df.loc[(music_df['band'] == 'radiohead_lollapalooza'), 'band'] = 'radiohead'


# Some specific rows to fix
#143
music_df.loc[music_df['id'] == 'f808ec4b46eb140fca6b9b73f6cbff76ee0e6641', 'band'] = 'neil young'
music_df.loc[music_df['id'] == 'f808ec4b46eb140fca6b9b73f6cbff76ee0e6641', 'date_right_after_band'] = False
#185
music_df.loc[music_df['id'] == 'e059e8d39da4b75145b1e434b0569df9c8f59367', 'band'] = 'the ramones'
music_df.loc[music_df['id'] == 'e059e8d39da4b75145b1e434b0569df9c8f59367', 'date_right_after_band'] = False
#192
music_df.loc[music_df['id'] == 'ff3fe93793664814e48139fbd51022e73347ec86', 'band'] = 'the ramones'
music_df.loc[music_df['id'] == 'ff3fe93793664814e48139fbd51022e73347ec86', 'date_right_after_band'] = False
#234
music_df.loc[music_df['id'] == 'ebb2e1757bdb2a01716acefca2697fd718d9a8c1', 'band'] = 'rush'
music_df.loc[music_df['id'] == 'ebb2e1757bdb2a01716acefca2697fd718d9a8c1', 'date_right_after_band'] = False
#244
music_df.loc[music_df['id'] == 'c2b7c10aa42aa6f55c8537ece72ed5726ae6b1b3', 'band'] = 'deadco'
music_df.loc[music_df['id'] == 'c2b7c10aa42aa6f55c8537ece72ed5726ae6b1b3', 'date_right_after_band'] = False
#781
music_df.loc[music_df['id'] == '587888.html', 'band'] = 'phish-sleeptrain'
music_df.loc[music_df['id'] == '587888.html', 'date_right_after_band'] = False
#840
music_df.loc[music_df['id'] == '565503.html', 'band'] = 'the cure'
music_df.loc[music_df['id'] == '565503.html', 'date_right_after_band'] = False
#945
music_df.loc[music_df['id'] == '565650.html', 'band'] = 'maku'
music_df.loc[music_df['id'] == '565650.html', 'date_right_after_band'] = False
#1046
music_df.loc[music_df['id'] == '565793.html', 'band'] = 'gordon gatton'
music_df.loc[music_df['id'] == '565793.html', 'date_right_after_band'] = False
#1087
music_df.loc[music_df['id'] == '565852.html', 'band'] = 'the obsessed'
music_df.loc[music_df['id'] == '565852.html', 'date_right_after_band'] = False
#1115
music_df.loc[music_df['id'] == '565891.html', 'band'] = 'nickcave'
#1139
music_df.loc[music_df['id'] == '565923.html', 'band'] = 'vm'
music_df.loc[music_df['id'] == '565923.html', 'date_right_after_band'] = False
#1149
music_df.loc[music_df['id'] == '565934.html', 'band'] = 'l-ficher-g-baton'
music_df.loc[music_df['id'] == '565934.html', 'date_right_after_band'] = False
#1162
music_df.loc[music_df['id'] == '565954.html', 'band'] = 'ehg'
music_df.loc[music_df['id'] == '565954.html', 'date_right_after_band'] = False
#1233
music_df.loc[music_df['id'] == '566050.html', 'band'] = 'black sabbath'
#1245
music_df.loc[music_df['id'] == '566066.html', 'band'] = 'bob dylan'
music_df.loc[music_df['id'] == '566066.html', 'date_right_after_band'] = False
#1277
music_df.loc[music_df['id'] == '566114.html', 'band'] = 'mavis staples'
#1288
music_df.loc[music_df['id'] == '566127.html', 'band'] = 'the jam'
music_df.loc[music_df['id'] == '566127.html', 'date_right_after_band'] = False
#1295
music_df.loc[music_df['id'] == '566136.html', 'band'] = 'karma to burn'
music_df.loc[music_df['id'] == '566136.html', 'date_right_after_band'] = False
#1301
music_df.loc[music_df['id'] == '566144.html', 'band'] = 'the jam'
music_df.loc[music_df['id'] == '566144.html', 'date_right_after_band'] = False
#1312
music_df.loc[music_df['id'] == '566155.html', 'band'] = 'the jam'
music_df.loc[music_df['id'] == '566155.html', 'date_right_after_band'] = False
#1314
music_df.loc[music_df['id'] == '566157.html', 'band'] = 'the jam'
music_df.loc[music_df['id'] == '566157.html', 'date_right_after_band'] = False
#1355
music_df.loc[music_df['id'] == '566208.html', 'band'] = 'danish radio big band'
music_df.loc[music_df['id'] == '566208.html', 'date_right_after_band'] = False
#1377
music_df.loc[music_df['id'] == '566234.html', 'band'] = 'jam'
#1393
music_df.loc[music_df['id'] == '566252.html', 'band'] = 'casting crowns'
#1403
music_df.loc[music_df['id'] == '566264.html', 'band'] = 'paul mccartney'
music_df.loc[music_df['id'] == '566264.html', 'date_right_after_band'] = False
#1414
music_df.loc[music_df['id'] == '566278.html', 'band'] = 'dizzy mizz lizzy'
music_df.loc[music_df['id'] == '566278.html', 'date_right_after_band'] = False
#1533
music_df.loc[music_df['id'] == '566439.html', 'band'] = 'hw'
music_df.loc[music_df['id'] == '566439.html', 'date_right_after_band'] = False
#1596
music_df.loc[music_df['id'] == '566526.html', 'band'] = 'jam'
music_df.loc[music_df['id'] == '566526.html', 'date_right_after_band'] = False
#1683
music_df.loc[music_df['id'] == '566657.html', 'band'] = 'van morrison'
music_df.loc[music_df['id'] == '566657.html', 'date_right_after_band'] = False
#1706
music_df.loc[music_df['id'] == '566685.html', 'band'] = 'glen hansard'
music_df.loc[music_df['id'] == '566685.html', 'date_right_after_band'] = False
#1737
music_df.loc[music_df['id'] == '566734.html', 'band'] = 'peter &amp; the test tube babies'
music_df.loc[music_df['id'] == '566734.html', 'date_right_after_band'] = False
#1759
music_df.loc[music_df['id'] == '566756.html', 'band'] = 'marillion'
music_df.loc[music_df['id'] == '566756.html', 'date_right_after_band'] = False
#1784
music_df.loc[music_df['id'] == '566783.html', 'band'] = 'demiricous'
music_df.loc[music_df['id'] == '566783.html', 'date_right_after_band'] = False
#1790
music_df.loc[music_df['id'] == '566789.html', 'band'] = 'the clash'
music_df.loc[music_df['id'] == '566789.html', 'date_right_after_band'] = False
#1796
music_df.loc[music_df['id'] == '566798.html', 'band'] = 'keith jarrett'
music_df.loc[music_df['id'] == '565954.html', 'date_right_after_band'] = False
#1808
music_df.loc[music_df['id'] == '566810.html', 'band'] = 'lucie vitkova-ingelfritz'
music_df.loc[music_df['id'] == '566810.html', 'date_right_after_band'] = False
#1811
music_df.loc[music_df['id'] == '566814.html', 'band'] = 'mjarre'
#1869
music_df.loc[music_df['id'] == '566896.html', 'mics_etc'] = 'macca'
#1875  
music_df.loc[music_df['id'] == '566903.html', 'band'] = 'dixie chicks'
music_df.loc[music_df['id'] == '566903.html', 'date_right_after_band'] = False
#1933
music_df.loc[music_df['id'] == '566973.html', 'band'] = 'chess smith trio'
music_df.loc[music_df['id'] == '566973.html', 'date_right_after_band'] = False
#1984
music_df.loc[music_df['id'] == '567047.html', 'band'] = 'charlie cunningham'
music_df.loc[music_df['id'] == '567047.html', 'date_right_after_band'] = False
#2040
music_df.loc[music_df['id'] == '567120.html', 'band'] = 'neil young'
music_df.loc[music_df['id'] == '567120.html', 'date_right_after_band'] = False
#2270
music_df.loc[music_df['id'] == '567402.html', 'band'] = 'nickcave'
music_df.loc[music_df['id'] == '567402.html', 'date_right_after_band'] = False
#2053
music_df.loc[music_df['id'] == '567140.html', 'band'] = 'kiss'
music_df.loc[music_df['id'] == '567140.html', 'date_right_after_band'] = False
#2092
music_df.loc[music_df['id'] == '567194.html', 'band'] = 'acdc'
music_df.loc[music_df['id'] == '567194.html', 'date_right_after_band'] = False
#2099
music_df.loc[music_df['id'] == '567202.html', 'band'] = 'lk'
#2200
music_df.loc[music_df['id'] == '567318.html', 'band'] = 'vm'
music_df.loc[music_df['id'] == '567318.html', 'date_right_after_band'] = False
#2270
music_df.loc[music_df['id'] == '567402.html', 'band'] = 'nickcave'
music_df.loc[music_df['id'] == '567402.html', 'date_right_after_band'] = False
#2374
music_df.loc[music_df['id'] == '567536.html', 'band'] = ''
music_df.loc[music_df['id'] == '565650.html', 'date_right_after_band'] = False
#2516 
music_df.loc[music_df['id'] == '567720.html', 'band'] = 'david bowie'
music_df.loc[music_df['id'] == '567720.html', 'date_right_after_band'] = False
#2558
music_df.loc[music_df['id'] == '567771.html', 'band'] = 'daywave'
music_df.loc[music_df['id'] == '567771.html', 'date_right_after_band'] = False
#2569
music_df.loc[music_df['id'] == '567795.html', 'band'] = 'rob brown quartet'
music_df.loc[music_df['id'] == '567795.html', 'date_right_after_band'] = False
#2634
music_df.loc[music_df['id'] == '567879.html', 'band'] = 'modern baseball'
music_df.loc[music_df['id'] == '567879.html', 'date_right_after_band'] = False
#2823
music_df.loc[music_df['id'] == '127023.html', 'band'] = 'neil young'
#2828
music_df.loc[music_df['id'] == '127033.html', 'band'] = 'sm pf'
music_df.loc[music_df['id'] == '127033.html', 'date_right_after_band'] = False

## SECOND ROUND found by Jordan 3/27/2018
#186
music_df.loc[music_df['id'] == '5a3b55bc30d8dabb1023d442759c34dd70edebb1', 'band'] = ''
music_df.loc[music_df['id'] == '5a3b55bc30d8dabb1023d442759c34dd70edebb1', 'date_right_after_band'] = False
#636
music_df.loc[music_df['id'] == '587838.html', 'band'] = ''
music_df.loc[music_df['id'] == '587838.html', 'date_right_after_band'] = False
#778
music_df.loc[music_df['id'] == '565641.html', 'band'] = ''
music_df.loc[music_df['id'] == '565641.html', 'date_right_after_band'] = False
#950
music_df.loc[music_df['id'] == '566104.html', 'band'] = 'bob dylan'
music_df.loc[music_df['id'] == '566104.html', 'date_right_after_band'] = False
#1002
music_df.loc[music_df['id'] == '566232.html', 'band'] = ''
music_df.loc[music_df['id'] == '566232.html', 'date_right_after_band'] = False
#1793
music_df.loc[music_df['id'] == '127205.html', 'band'] = ''
music_df.loc[music_df['id'] == '127205.html', 'date_right_after_band'] = False
#881
music_df.loc[music_df['id'] == '565901.html', 'band'] = 'whitford'
music_df.loc[music_df['id'] == '565901.html', 'date_right_after_band'] = False
#1181
music_df.loc[music_df['id'] == '566727.html', 'band'] = 'buben'
music_df.loc[music_df['id'] == '566727.html', 'date_right_after_band'] = False
#1374
music_df.loc[music_df['id'] == '567234.html', 'band'] = 'gnr'
music_df.loc[music_df['id'] == '567234.html', 'date_right_after_band'] = False
#1587
music_df.loc[music_df['id'] == '567731.html', 'band'] = 'the brew'
music_df.loc[music_df['id'] == '567731.html', 'date_right_after_band'] = False

#music_df[['id', 'filename', 'band', 'date_right_after_band']].head(3)
print('Parsed band abbrevs for {0:,.0f} filenames (out of {1:,.0f} total).'.format(len(music_df[music_df['band'] != '']), len(music_df)))
print('Found {0:,.0f} distinct band abbrevs.'.format(len(music_df['band'][music_df['band'] != ''].drop_duplicates())))

Parsed band abbrevs for 1,543 filenames (out of 1,803 total).
Found 758 distinct band abbrevs.


In [6]:
# Check band names vs. list of canonical abbreviations
music_df['abbrev_type'] = 'xxx'
music_df.loc[music_df['band'].isin(canonical_abbrevs), 'abbrev_type'] = 'canonical'
# Check band names vs. list of variant abbreviations
music_df.loc[music_df['band'].isin(variant_abbrevs), 'abbrev_type'] = 'variant'
# No band name
music_df.loc[music_df['band'] == '', 'abbrev_type'] = 'blank'

# Band names used incorrect abbreviations
incorrect = ['phish', 'widespreadpanic', 'jg+ms', 'deadco', 'deadandco', 'dc', 'jgms']
incorrect += ['dead and company', 'dead.and.co', 'dead.and.co', 'dac', 'dandc']
incorrect += ['deadco-', 'dead&co', 'dead&amp;co', 'dead&amp;company', 'deadandcompany']
incorrect += ['deadco.irvine.', 'deadfeat', 'gov`t mule']
music_df.loc[music_df['band'].isin(incorrect), 'abbrev_type'] = 'incorrect'

# Band names not in abbreviation list
notfound = ['loslobos', 'deanmonkey&amp;thedropouts', 'ttb', 'crb', 'deadandcompany', 'ny', 'gsbg', 'bd']
notfound += ['jefferson starship', 'carlton, larry -', 'the who', 's marriott', 'unknown artist']
notfound += ['tal wilkenfeld', 'ph&amp;tl', 'sb', 'radiohead', 'theclash_', 'primus']

vc = music_df['band'][music_df['band'] != ''].value_counts()
# If a band name appears 4 or fewer times, assume it's a misc band not found in the list.
for x in (vc[(vc<5)].index.sort_values().tolist()):
    if ((x not in canonical_abbrevs) and (x not in variant_abbrevs) and (x not in notfound) and (x not in incorrect)):
        notfound.append(x)
        
notfound += ['u2', 'the jam', 'theclash', 'carlton, larry']
music_df.loc[music_df['band'].isin(notfound), 'abbrev_type'] = 'notfound'

#music_df[['id', 'filename', 'band', 'date_right_after_band', 'abbrev_type']][music_df['abbrev_type'] == 'xxx'].sample(10)
music_df.groupby('abbrev_type').size()

abbrev_type
blank        260 
canonical    329 
incorrect    107 
notfound     1074
variant      33  
dtype: int64

In [7]:
# Get Jeremy's list of bands
bands_df = pandas.read_csv('bands_verified.csv', index_col=0, low_memory=False)#, dtype={'band': 'S1024', 'band_or_not': 'S6', 'canonical_abbreviation': 'S16', 'notes': 'S4096'})

bands_df.index = pandas.to_numeric(bands_df.index)

for thiscol in ['band', 'canonical_abbreviation', 'non_canonical', 'notes']:
    bands_df[thiscol] = bands_df[thiscol].apply(lambda x: str(x)[1:-1])

bands_df['band_or_not'] = bands_df['band_or_not'].apply(lambda x: x.lower())


music_df = music_df.merge(bands_df, how='left', on='band')
#music_df[music_df['band_or_not'] == 'yes'].sample(10)

music_df.loc[(music_df['abbrev_type'] == 'notfound') & (music_df['band_or_not'] == 'yes'), 'abbrev_type'] = 'jeremy'
music_df.loc[(music_df['abbrev_type'] == 'notfound') & (music_df['band_or_not'] == 'maybe'), 'abbrev_type'] = 'notband'
music_df.loc[(music_df['abbrev_type'] == 'notfound') & (music_df['band_or_not'] == 'no'), 'abbrev_type'] = 'notband'

music_df = music_df.drop(['band_or_not', 'canonical_abbreviation', 'non_canonical'], axis=1)
music_df = music_df.rename(columns={'notes': 'jeremy_band_finding_notes'})
music_df['jeremy_band_finding_notes'] = music_df['jeremy_band_finding_notes'].fillna('')

music_df.groupby('abbrev_type').size().sort_values(ascending=False)

abbrev_type
jeremy       896
canonical    329
blank        260
notband      178
incorrect    107
variant      33 
dtype: int64

In [8]:
date1 = re.compile('\d+[\-\/\ —]+\d+[\-\/\ —]+\d+')
date2 = re.compile('(\d{8})')
date3 = re.compile('(19|20)\d{2}[-]\D+[-]\d{1,2}')

music_df['parsed_date'] = pandas.to_datetime(np.nan)
music_df['mics_etc'] = ''
music_df['date_format'] = ''

# Parse everything after the first digit for analysis of dates
music_df['after_first_digit'] = music_df['remainder'].apply(lambda x: x[re.search('\d', x).start():len(x)])

# Fix for u2 since they have a number as their band name
music_df.loc[music_df['band'] == 'u2', 
            'after_first_digit'] = music_df['after_first_digit'][
    music_df['band'] == 'u2'].apply(
lambda x: x[re.search('^2[\s\_\-]*', x).end():len(x)])

# Parse date as yyyy-mm-dd
music_df['full_date'] = pandas.to_datetime(music_df['after_first_digit'].apply(lambda x: x[0:10]), format='%Y-%m-%d', errors='coerce').dropna()
music_df['full_date_mics_etc'] = music_df['after_first_digit'].apply(lambda x: x[10:])
music_df.loc[(music_df['date_format'] == '') & (music_df['full_date'].notnull()), 'date_format'] = 'yyyy-mm-dd'
music_df.loc[music_df['date_format'] == 'yyyy-mm-dd', 'parsed_date'] = music_df['full_date'][music_df['date_format'] == 'yyyy-mm-dd']
music_df.loc[music_df['date_format'] == 'yyyy-mm-dd', 'mics_etc'] = music_df['full_date_mics_etc'][music_df['date_format'] == 'yyyy-mm-dd']

# Parse date as yy-mm-dd
music_df['yy_date'] = pandas.to_datetime(music_df['after_first_digit'].apply(lambda x: x[0:8]), format='%y-%m-%d', errors='coerce').dropna()
music_df['yy_date_mics_etc'] = music_df['after_first_digit'].apply(lambda x: x[8:])
music_df.loc[(music_df['date_format'] == '') & (music_df['yy_date'].notnull()), 'date_format'] = 'yy-mm-dd'
music_df.loc[music_df['date_format'] == 'yy-mm-dd', 'parsed_date'] = music_df['yy_date'][music_df['date_format'] == 'yy-mm-dd']
music_df.loc[music_df['date_format'] == 'yy-mm-dd', 'mics_etc'] = music_df['yy_date_mics_etc'][music_df['date_format'] == 'yy-mm-dd']

# Parse date as date1 format
music_df['date1_date'] = pandas.to_datetime(music_df['after_first_digit'][(music_df['date_format'] == '') & 
                             (music_df['after_first_digit'].apply(lambda x: re.search(date1, x)))
                             ].apply(lambda x: x[re.search(date1, x).start():re.search(date1, x).end()]), errors='coerce')
music_df['date1_date_mics_etc'] = music_df['after_first_digit'][(music_df['date_format'] == '') & 
                             (music_df['after_first_digit'].apply(lambda x: re.search(date1, x)))
                             ].apply(lambda x: x[re.search(date1, x).end():])
music_df.loc[(music_df['date_format'] == '') & (music_df['date1_date'].notnull()), 'date_format'] = 'date1'
music_df.loc[music_df['date_format'] == 'date1', 'parsed_date'] = music_df['date1_date']
music_df.loc[music_df['date_format'] == 'date1', 'mics_etc'] = music_df['date1_date_mics_etc']

# Parse date as date2 format
music_df['date2_date'] = pandas.to_datetime(music_df['after_first_digit'][(music_df['date_format'] == '') & 
                             (music_df['after_first_digit'].apply(lambda x: re.search(date2, x)))
                             ].apply(lambda x: x[re.search(date2, x).start():re.search(date2, x).end()]), errors='coerce')
music_df['date2_date_mics_etc'] = music_df['after_first_digit'][(music_df['date_format'] == '') & 
                             (music_df['after_first_digit'].apply(lambda x: re.search(date2, x)))
                             ].apply(lambda x: x[re.search(date2, x).end():])
music_df.loc[(music_df['date_format'] == '') & (music_df['date2_date'].notnull()), 'date_format'] = 'date2'
music_df.loc[music_df['date_format'] == 'date2', 'parsed_date'] = music_df['date2_date']
music_df.loc[music_df['date_format'] == 'date2', 'mics_etc'] = music_df['date2_date_mics_etc']

# Parse date as date3 format
music_df['date3_date'] = pandas.to_datetime(music_df['after_first_digit'][(music_df['date_format'] == '') & 
                             (music_df['after_first_digit'].apply(lambda x: re.search(date3, x)))
                             ].apply(lambda x: x[re.search(date3, x).start():re.search(date3, x).end()]), errors='coerce')
music_df['date3_date_mics_etc'] = music_df['after_first_digit'][(music_df['date_format'] == '') & 
                             (music_df['after_first_digit'].apply(lambda x: re.search(date3, x)))
                             ].apply(lambda x: x[re.search(date3, x).end():])
music_df.loc[(music_df['date_format'] == '') & (music_df['date3_date'].notnull()), 'date_format'] = 'date3'
music_df.loc[music_df['date_format'] == 'date3', 'parsed_date'] = music_df['date3_date']
music_df.loc[music_df['date_format'] == 'date3', 'mics_etc'] = music_df['date3_date_mics_etc']


# Remove interim columns
music_df = music_df.drop(
    ['full_date', 'full_date_mics_etc', 'yy_date', 'yy_date_mics_etc', 
     'date1_date', 'date1_date_mics_etc', 'date2_date', 'date2_date_mics_etc',
     'date3_date', 'date3_date_mics_etc'], axis=1)


#
#music_df[['filename', 'after_first_digit', 'parsed_date', 'mics_etc', 'date_format']][music_df['date_format'] == '']
                                                         
#music_df[['filename', 'after_first_digit', 'parsed_date', 'mics_etc', 'date_format']
#        ]
music_df.groupby('date_format').size()

date_format
              340 
date1         69  
date2         96  
date3         5   
yy-mm-dd      123 
yyyy-mm-dd    1170
dtype: int64

In [27]:
music_df[['id', 'filename']][music_df['date_format'] == '']

music_df['date_by_hand'] = pandas.to_datetime(np.nan)

#82
music_df.loc[music_df['id'] == '3c39e32bf0918ec3190ecda08442eba0d8b2faa8', 
             'date_by_hand'] = pandas.to_datetime('1997-06-22')
music_df.loc[music_df['id'] == '3c39e32bf0918ec3190ecda08442eba0d8b2faa8', 
             'date_by_hand_mics_etc'] = 'beausolield1tr01'
#191
music_df.loc[music_df['id'] == 'c2b7c10aa42aa6f55c8537ece72ed5726ae6b1b3', 
             'date_by_hand'] = pandas.to_datetime('2016-07-26')
music_df.loc[music_df['id'] == 'c2b7c10aa42aa6f55c8537ece72ed5726ae6b1b3', 
             'date_by_hand_mics_etc'] = '.01'
#218
music_df.loc[music_df['id'] == '587337.html', 
             'date_by_hand'] = pandas.to_datetime('2016-06-03')
music_df.loc[music_df['id'] == '587337.html', 
             'date_by_hand_mics_etc'] = '01'
#219
music_df.loc[music_df['id'] == '587339.html', 
             'date_by_hand'] = pandas.to_datetime('2016-02-19')
#316
music_df.loc[music_df['id'] == '587454.html', 
             'date_by_hand'] = pandas.to_datetime('2016-01-30')
music_df.loc[music_df['id'] == '587454.html', 
             'date_by_hand_mics_etc'] = '01'
#409
music_df.loc[music_df['id'] == '587572.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-09')
music_df.loc[music_df['id'] == '587572.html', 
             'date_by_hand_mics_etc'] = '01'
#421
music_df.loc[music_df['id'] == '587588.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-10')
music_df.loc[music_df['id'] == '587588.html', 
             'date_by_hand_mics_etc'] = '01'
#438
music_df.loc[music_df['id'] == '587606.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-11')
music_df.loc[music_df['id'] == '587606.html', 
             'date_by_hand_mics_etc'] = '_t01'
#467
music_df.loc[music_df['id'] == '587640.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-08')
music_df.loc[music_df['id'] == '587640.html', 
             'date_by_hand_mics_etc'] = '01'
#588
music_df.loc[music_df['id'] == '587786.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-15')
music_df.loc[music_df['id'] == '587786.html', 
             'date_by_hand_mics_etc'] = ' george,wa t15 crosseyed and painless'
#676
music_df.loc[music_df['id'] == '587888.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-22')
music_df.loc[music_df['id'] == '587888.html', 
             'date_by_hand_mics_etc'] = '.01'
#749
music_df.loc[music_df['id'] == '565561.html', 
             'date_by_hand'] = pandas.to_datetime('2011-09-06')
#786
music_df.loc[music_df['id'] == '565664.html', 
             'date_by_hand'] = pandas.to_datetime('2016-05-31')
music_df.loc[music_df['id'] == '565664.html', 
             'date_by_hand_mics_etc'] = ' 210'
#824
music_df.loc[music_df['id'] == '565762.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-03')
#833
music_df.loc[music_df['id'] == '565781.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-03')
#837
music_df.loc[music_df['id'] == '565793.html', 
             'date_by_hand'] = pandas.to_datetime('1981-06-27')
music_df.loc[music_df['id'] == '565793.html', 
             'date_by_hand_mics_etc'] = '01'
#876
music_df.loc[music_df['id'] == '565891.html', 
             'date_by_hand'] = pandas.to_datetime('1989-02-18')
music_df.loc[music_df['id'] == '565891.html', 
             'date_by_hand_mics_etc'] = '_d6_909_t01'

#891
music_df.loc[music_df['id'] == '565923.html', 
             'date_by_hand'] = pandas.to_datetime('1997-07-05')
music_df.loc[music_df['id'] == '565923.html', 
             'date_by_hand_mics_etc'] = '_disc 02 track 02'
#903
music_df.loc[music_df['id'] == '565954.html', 
             'date_by_hand'] = pandas.to_datetime('2016-04-29')
music_df.loc[music_df['id'] == '565954.html', 
             'date_by_hand_mics_etc'] = '- t01'
#909
music_df.loc[music_df['id'] == '565970.html', 
             'date_by_hand'] = pandas.to_datetime('2016-06-06')
music_df.loc[music_df['id'] == '565970.html', 
             'date_by_hand_mics_etc'] = ' destinies entwined'
#937
music_df.loc[music_df['id'] == '566066.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-07')
music_df.loc[music_df['id'] == '566066.html', 
             'date_by_hand_mics_etc'] = ' - t01'
#950
music_df.loc[music_df['id'] == '566104.html', 
             'date_by_hand'] = pandas.to_datetime('2016-06-25')
music_df.loc[music_df['id'] == '566104.html', 
             'date_by_hand_mics_etc'] = ' - t01'
#954
music_df.loc[music_df['id'] == '566114.html', 
             'date_by_hand'] = pandas.to_datetime('2016-06-25')
music_df.loc[music_df['id'] == '566114.html', 
             'date_by_hand_mics_etc'] = ' - t01'
#957
music_df.loc[music_df['id'] == '566127.html', 
             'date_by_hand'] = pandas.to_datetime('1980-03-07')
music_df.loc[music_df['id'] == '566127.html', 
             'date_by_hand_mics_etc'] = ' 16'
#967
music_df.loc[music_df['id'] == '566148.html', 
             'date_by_hand'] = pandas.to_datetime('2012-11-06')
music_df.loc[music_df['id'] == '566148.html', 
             'date_by_hand_mics_etc'] = '_berlin_16_tr01'
#971
music_df.loc[music_df['id'] == '566157.html', 
             'date_by_hand'] = pandas.to_datetime('1979-11-24')
music_df.loc[music_df['id'] == '566157.html', 
             'date_by_hand_mics_etc'] = '_tr15'
#991
music_df.loc[music_df['id'] == '566210.html', 
             'date_by_hand'] = pandas.to_datetime('2016-01-08')
#996
music_df.loc[music_df['id'] == '566219.html', 
             'date_by_hand'] = pandas.to_datetime('1990-02-17')
music_df.loc[music_df['id'] == '566219.html', 
             'date_by_hand_mics_etc'] = '_d6_909_t01'
#999
music_df.loc[music_df['id'] == '566223.html', 
             'date_by_hand'] = pandas.to_datetime('1988-07-09')
music_df.loc[music_df['id'] == '566223.html', 
             'date_by_hand_mics_etc'] = '_07'
#1004
music_df.loc[music_df['id'] == '566234.html', 
             'date_by_hand'] = pandas.to_datetime('1979-11-18')
music_df.loc[music_df['id'] == '566234.html', 
             'date_by_hand_mics_etc'] = '_15'
#1014
music_df.loc[music_df['id'] == '566252.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-09')
music_df.loc[music_df['id'] == '566252.html', 
             'date_by_hand_mics_etc'] = '01'
#1020
music_df.loc[music_df['id'] == '566264.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-10')
music_df.loc[music_df['id'] == '566264.html', 
             'date_by_hand_mics_etc'] = ' - t01'
#1040
music_df.loc[music_df['id'] == '566331.html', 
             'date_by_hand'] = pandas.to_datetime('2012-11-06')
music_df.loc[music_df['id'] == '566331.html', 
             'date_by_hand_mics_etc'] = '_berlin_16_tr01'
#1154
music_df.loc[music_df['id'] == '566657.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-07')
music_df.loc[music_df['id'] == '566657.html', 
             'date_by_hand_mics_etc'] = ' (biddie)_track 17'
#1166
music_df.loc[music_df['id'] == '566680.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-11')
music_df.loc[music_df['id'] == '566680.html', 
             'date_by_hand_mics_etc'] = ' 115'
#1209
music_df.loc[music_df['id'] == '566783.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-15')
music_df.loc[music_df['id'] == '566783.html', 
             'date_by_hand_mics_etc'] = ' - t01'
#1297
music_df.loc[music_df['id'] == '567003.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-17')
music_df.loc[music_df['id'] == '567003.html', 
             'date_by_hand_mics_etc'] = '_1851'
#1309
music_df.loc[music_df['id'] == '567053.html', 
             'date_by_hand'] = pandas.to_datetime('1981-08-20')
music_df.loc[music_df['id'] == '567053.html', 
             'date_by_hand_mics_etc'] = 'scruzcivic'
#1335
music_df.loc[music_df['id'] == '567140.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-20')
#1339
music_df.loc[music_df['id'] == '567146.html', 
             'date_by_hand'] = pandas.to_datetime('2013-04-01')
#1366
music_df.loc[music_df['id'] == '567220.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-02')
music_df.loc[music_df['id'] == '567220.html', 
             'date_by_hand_mics_etc'] = ')'
#1421
music_df.loc[music_df['id'] == '567318.html', 
             'date_by_hand'] = pandas.to_datetime('2011-09-02')
music_df.loc[music_df['id'] == '567318.html', 
             'date_by_hand_mics_etc'] = '_disc 01 track 04'
#1445
music_df.loc[music_df['id'] == '567376.html', 
             'date_by_hand'] = pandas.to_datetime('1975-08-17')
music_df.loc[music_df['id'] == '567376.html', 
             'date_by_hand_mics_etc'] = '_04'
#1468
music_df.loc[music_df['id'] == '567438.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-12')
music_df.loc[music_df['id'] == '567438.html', 
             'date_by_hand_mics_etc'] = '.001'
#1498
music_df.loc[music_df['id'] == '567513.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-22')
music_df.loc[music_df['id'] == '567513.html', 
             'date_by_hand_mics_etc'] = '_chatham_track_12'
#1523
music_df.loc[music_df['id'] == '567572.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-22')
music_df.loc[music_df['id'] == '567572.html', 
             'date_by_hand_mics_etc'] = '_chatham_track_18'
#1526
music_df.loc[music_df['id'] == '567578.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-08')
music_df.loc[music_df['id'] == '567578.html', 
             'date_by_hand_mics_etc'] = '-trembling bells aud01'
#1545
music_df.loc[music_df['id'] == '567639.html', 
             'date_by_hand'] = pandas.to_datetime('1975-01-03')
music_df.loc[music_df['id'] == '567639.html', 
             'date_by_hand_mics_etc'] = ' u t'
#1551
music_df.loc[music_df['id'] == '567649.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-23')
music_df.loc[music_df['id'] == '567649.html', 
             'date_by_hand_mics_etc'] = '_rochester_track_20'
#1554
music_df.loc[music_df['id'] == '567662.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-23')
music_df.loc[music_df['id'] == '567662.html', 
             'date_by_hand_mics_etc'] = '_rochesterr_track_04'
#1557
music_df.loc[music_df['id'] == '567671.html', 
             'date_by_hand'] = pandas.to_datetime('1989-11-19')
music_df.loc[music_df['id'] == '567671.html', 
             'date_by_hand_mics_etc'] = '_07'
#1598
music_df.loc[music_df['id'] == '567750.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-23')
music_df.loc[music_df['id'] == '567750.html', 
             'date_by_hand_mics_etc'] = '_rochesterr_track_07'
#1606
music_df.loc[music_df['id'] == '567790.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-26')
music_df.loc[music_df['id'] == '567790.html', 
             'date_by_hand_mics_etc'] = '.aud.mk4.petaluma.flac16 01'
#1634
music_df.loc[music_df['id'] == '567872.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-30')
music_df.loc[music_df['id'] == '567872.html', 
             'date_by_hand_mics_etc'] = '_07'
#1677
music_df.loc[music_df['id'] == '126880.html', 
             'date_by_hand_mics_etc'] = pandas.to_datetime('1988-10-01')

music_df['date_by_hand_mics_etc'] = music_df['date_by_hand_mics_etc'].fillna('')
#music_df[['id', 'filename']][
#    (music_df['date_by_hand'].isnull()) 
#    & (music_df['date_format'] == '')
#].loc[1757:]

music_df.loc[music_df['date_by_hand'].notnull(), 'date_format'] = 'by_hand'

music_df.loc[music_df['date_format'] == 'by_hand', 'parsed_date'] = music_df['date_by_hand']
music_df.loc[music_df['date_format'] == 'by_hand', 'mics_etc'] = music_df['date_by_hand_mics_etc']

music_df = music_df.drop(['date_by_hand', 'date_by_hand_mics_etc'], axis=1)

#music_df[['id', 'filename', 'date_format', 'parsed_date', 'mics_etc']][music_df['date_format'] == 'by_hand']
music_df.groupby('date_format').size()


date_format
              286 
by_hand       54  
date1         69  
date2         96  
date3         5   
yy-mm-dd      123 
yyyy-mm-dd    1170
dtype: int64

In [ ]:
print('\nStep 3a. Score = 4: canonical abbreviation, date right after band, yyyy-mm-dd date')
fours_df = fours_df.append(music_df.loc[(music_df['abbrev_type'] == 'canonical') & 
         (music_df['date_right_after_band']) &
         (music_df['full_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(fours_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 3b. canonical abbreviation, date NOT right after band, yyyy-mm-dd date (score = 3)')
threes_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'canonical') & 
         (~music_df['date_right_after_band']) &
         (music_df['full_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(threes_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 3c. variant abbreviation, date right after band, yyyy-mm-dd date (score = 3))')
threes_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'variant') & 
         (music_df['date_right_after_band']) &
         (music_df['full_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(threes_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 3d. variant abbreviation, date NOT right after band, yyyy-mm-dd date (score = 2)')
twos_df = twos_df.append(music_df.loc[(music_df['abbrev_type'] == 'variant') & 
         (~music_df['date_right_after_band']) &
         (music_df['full_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 3e. incorrect abbreviation, date right after band, yyyy-mm-dd date (score = 3)')
threes_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'incorrect') & 
         (music_df['date_right_after_band']) &
         (music_df['full_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(threes_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 3f. incorrect abbreviation, date NOT right after band, yyyy-mm-dd date (score = 2)')
twos_df = twos_df.append(music_df.loc[(music_df['abbrev_type'] == 'incorrect') & 
         (~music_df['date_right_after_band']) &
         (music_df['full_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 3g. Score = 4: unregistered band, date right after band, yyyy-mm-dd date')
fours_df = fours_df.append(music_df.loc[(music_df['abbrev_type'] == 'jeremy') & 
         (music_df['date_right_after_band']) &
         (music_df['full_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(fours_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 3h. unregistered band, date NOT right after band, yyyy-mm-dd date (score = 3)')
threes_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'jeremy') & 
         (~music_df['date_right_after_band']) &
         (music_df['full_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(threes_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 3i. not a real band, date right after band, yyyy-mm-dd date (score = 3)')
threes_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'notband') & 
         (music_df['date_right_after_band']) &
         (music_df['full_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(threes_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 3j. not a real band, date NOT right after band, yyyy-mm-dd date (score = 2)')
twos_df = twos_df.append(music_df.loc[(music_df['abbrev_type'] == 'notband') & 
         (~music_df['date_right_after_band']) &
         (music_df['full_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

In [ ]:
print('\nStep 4a. Canonical abbreviation, date right after band, yy-mm-dd date (score = 3)')
threes_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'canonical') & 
         (music_df['date_right_after_band']) &
         (music_df['yy_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(threes_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 4b. Canonical abbreviation, date NOT right after band, yy-mm-dd date (score = 2)')
twos_df = twos_df.append(music_df.loc[(music_df['abbrev_type'] == 'canonical') & 
         (~music_df['date_right_after_band']) &
         (music_df['yy_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 4c. variant abbreviation, date right after band, yy-mm-dd date (score=2)')
twos_df = twos_df.append(music_df.loc[(music_df['abbrev_type'] == 'variant') & 
         (music_df['date_right_after_band']) &
         (music_df['yy_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 4d. variant abbreviation, date NOT right after band, yy-mm-dd date (score = 1)')
ones_df = ones_df.append(music_df.loc[(music_df['abbrev_type'] == 'variant') & 
         (~music_df['date_right_after_band']) &
         (music_df['yy_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(ones_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 4e. incorrect abbreviation, date right after band, yy-mm-dd date (score = 2)')
twos_df = twos_df.append(music_df.loc[(music_df['abbrev_type'] == 'incorrect') & 
         (music_df['date_right_after_band']) &
         (music_df['yy_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 4f. incorrect abbreviation, date NOT right after band, yy-mm-dd date (score = 1)')
ones_df = ones_df.append(music_df.loc[(music_df['abbrev_type'] == 'incorrect') & 
         (~music_df['date_right_after_band']) &
         (music_df['yy_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(ones_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 4g. unregistered band, date right after band, yy-mm-dd date (score=3)')
threes_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'jeremy') & 
         (music_df['date_right_after_band']) &
         (music_df['yy_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(threes_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 4h. unregistered band, date NOT right after band, yy-mm-dd date (score = 2)')
twos_df = twos_df.append(music_df.loc[(music_df['abbrev_type'] == 'jeremy') & 
         (~music_df['date_right_after_band']) &
         (music_df['yy_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 4i. not a real band, date right after band, yy-mm-dd date (score = 2)')
twos_df = twos_df.append(music_df.loc[(music_df['abbrev_type'] == 'notband') & 
         (music_df['date_right_after_band']) &
         (music_df['yy_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 4j. not a real band, date NOT right after band, yy-mm-dd date (score = 1)')
ones_df = ones_df.append(music_df.loc[(music_df['abbrev_type'] == 'notband') & 
         (~music_df['date_right_after_band']) &
         (music_df['yy_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(ones_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

# JORDAN IS UP TO HERE!

In [ ]:
date1 = re.compile('\d+[\-\/\ —]+\d+[\-\/\ —]+\d+')
date2 = re.compile('(\d{8})')

music_df.loc[music_df['after_first_digit'].apply(lambda x: re.search(date1, x) != None), 
            'parsed_date'] = pandas.to_datetime(music_df['after_first_digit'][music_df['after_first_digit'].apply(lambda x: re.search(date1, x) != None)].apply(
lambda x: x[re.search(date1, x).start(): re.search(date1, x).end()]), errors='coerce')

music_df.loc[music_df['after_first_digit'].apply(lambda x: re.search(date1, x) != None), 
            'parsed_date_mics_etc'] = music_df['after_first_digit'][music_df['after_first_digit'].apply(lambda x: re.search(date1, x) != None)].apply(
lambda x: x[re.search(date1, x).end(): len(x)])

music_df['parsed_date_mics_etc2'] = music_df['parsed_date_mics_etc'].dropna().loc[
    music_df['parsed_date_mics_etc'].dropna().apply(
    lambda x: re.search(markers, x) != None)].apply(
lambda x: x[re.search(markers, x).end():len(x)])


music_df['date_format'] = ''
music_df.loc[music_df['parsed_date'].notnull(), 'date_format'] = '\d+[\-\/\ —]+\d+[\-\/\ —]+\d+'

print('\nStep 5a. canonical abbreviation, date has format \d+[\-\/\ —]+\d+[\-\/\ —]+\d+ (score = 3)')
threes_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'canonical') & 
         (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(threes_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 5b. variant abbreviation, date has format \d+[\-\/\ —]+\d+[\-\/\ —]+\d+ (score = 2)')
twos_df = twos_df.append(music_df.loc[(music_df['abbrev_type'] == 'variant') & 
         (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 5c. incorrect abbreviation, date has format \d+[\-\/\ —]+\d+[\-\/\ —]+\d+ (score = 2)')
twos_df = twos_df.append(music_df.loc[(music_df['abbrev_type'] == 'incorrect') & 
         (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 5d. band abbreviation not in etree list, date has format \d+[\-\/\ —]+\d+[\-\/\ —]+\d+ (score = 3)')
threes_df = threes_df.append(music_df.loc[(music_df['abbrev_type'] == 'notfound') & 
         (music_df['parsed_date'].notnull())])
music_df = music_df.loc[~music_df.index.isin(threes_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

In [ ]:
music_df['date_by_hand'] = pandas.to_datetime(np.nan)
music_df['mics_etc'] = ''
#100
music_df.loc[music_df['id'] == '3c39e32bf0918ec3190ecda08442eba0d8b2faa8', 'date_by_hand'] = pandas.to_datetime('1997-06-22')
music_df.loc[music_df['id'] == '3c39e32bf0918ec3190ecda08442eba0d8b2faa8', 'mics_etc'] = 'beausolield1tr01'
#244
music_df.loc[music_df['id'] == 'c2b7c10aa42aa6f55c8537ece72ed5726ae6b1b3', 'date_by_hand'] = pandas.to_datetime('2016-07-26')
music_df.loc[music_df['id'] == 'c2b7c10aa42aa6f55c8537ece72ed5726ae6b1b3', 'mics_etc'] = 'irvine'
#281
music_df.loc[music_df['id'] == '587337.html', 'date_by_hand'] = pandas.to_datetime('2016-06-03')
music_df.loc[music_df['id'] == '587337.html', 'mics_etc'] = '01'
#282
music_df.loc[music_df['id'] == '587339.html', 'date_by_hand'] = pandas.to_datetime('2016-02-19')
music_df.loc[music_df['id'] == '587339.html', 'mics_etc'] = '01'
#390
music_df.loc[music_df['id'] == '587454.html', 'date_by_hand'] = pandas.to_datetime('2016-01-30')
music_df.loc[music_df['id'] == '587454.html', 'mics_etc'] = '01'
#492
music_df.loc[music_df['id'] == '587572.html', 'date_by_hand'] = pandas.to_datetime('2016-07-09')
music_df.loc[music_df['id'] == '587572.html', 'mics_etc'] = '01'
#505
music_df.loc[music_df['id'] == '587588.html', 'date_by_hand'] = pandas.to_datetime('2016-07-09')
music_df.loc[music_df['id'] == '587588.html', 'mics_etc'] = '01'
#523
music_df.loc[music_df['id'] == '587606.html', 'date_by_hand'] = pandas.to_datetime('2016-07-11')
music_df.loc[music_df['id'] == '587606.html', 'mics_etc'] = '01'
#556
music_df.loc[music_df['id'] == '587640.html', 'date_by_hand'] = pandas.to_datetime('2016-07-08')
music_df.loc[music_df['id'] == '587640.html', 'mics_etc'] = '01'
#686
music_df.loc[music_df['id'] == '587786.html', 'date_by_hand'] = pandas.to_datetime('2016-07-15')
music_df.loc[music_df['id'] == '587786.html', 'mics_etc'] = 'george,wa t15 crosseyed and painless'
#781
music_df.loc[music_df['id'] == '587888.html', 'date_by_hand'] = pandas.to_datetime('2016-07-22')
music_df.loc[music_df['id'] == '587888.html', 'mics_etc'] = 'cvca'
#881
music_df.loc[music_df['id'] == '565561.html', 'date_by_hand'] = pandas.to_datetime('2016-09-11')
music_df.loc[music_df['id'] == '565561.html', 'band'] = 'jimmy mo'
music_df.loc[music_df['id'] == '565561.html', 'mics_etc'] = 'mhile stor rte'
#955
music_df.loc[music_df['id'] == '565664.html', 'date_by_hand'] = pandas.to_datetime('2016-05-31')
music_df.loc[music_df['id'] == '565664.html', 'mics_etc'] = '210'
#1026
music_df.loc[music_df['id'] == '565762.html', 'date_by_hand'] = pandas.to_datetime('2016-07-03')
music_df.loc[music_df['id'] == '565762.html', 'mics_etc'] = '05'
#1037
music_df.loc[music_df['id'] == '565781.html', 'date_by_hand'] = pandas.to_datetime('2016-07-03')
music_df.loc[music_df['id'] == '565781.html', 'mics_etc'] = '05'
#1046
music_df.loc[music_df['id'] == '565793.html', 'date_by_hand'] = pandas.to_datetime('1981-06-27')
music_df.loc[music_df['id'] == '565793.html', 'mics_etc'] = '01'
#1115
music_df.loc[music_df['id'] == '565891.html', 'date_by_hand'] = pandas.to_datetime('1989-02-18')
music_df.loc[music_df['id'] == '565891.html', 'mics_etc'] = 'd6_909_t01'
#1139
music_df.loc[music_df['id'] == '565923.html', 'date_by_hand'] = pandas.to_datetime('1997-07-05')
music_df.loc[music_df['id'] == '565923.html', 'mics_etc'] = 'hamburg disc 02 track 02'
#1162
music_df.loc[music_df['id'] == '565954.html', 'date_by_hand'] = pandas.to_datetime('2016-04-29')
music_df.loc[music_df['id'] == '565954.html', 'mics_etc'] = 't01'
#1174
music_df.loc[music_df['id'] == '565970.html', 'date_by_hand'] = pandas.to_datetime('2016-06-06')
music_df.loc[music_df['id'] == '565970.html', 'band'] = 'the waterboys'
music_df.loc[music_df['id'] == '565970.html', 'mics_etc'] = 'destinies entwined'
#1245
music_df.loc[music_df['id'] == '566066.html', 'date_by_hand'] = pandas.to_datetime('2016-07-06')
music_df.loc[music_df['id'] == '566066.html', 'mics_etc'] = 't01'
#1270
music_df.loc[music_df['id'] == '566104.html', 'date_by_hand'] = pandas.to_datetime('2016-06-25')
music_df.loc[music_df['id'] == '566104.html', 'band'] = 'bob dylan'
music_df.loc[music_df['id'] == '566104.html', 'mics_etc'] = 't01'
#1277
music_df.loc[music_df['id'] == '566114.html', 'date_by_hand'] = pandas.to_datetime('2016-06-25')
music_df.loc[music_df['id'] == '566114.html', 'mics_etc'] = 't01'
#1288
music_df.loc[music_df['id'] == '566127.html', 'date_by_hand'] = pandas.to_datetime('1980-03-07')
music_df.loc[music_df['id'] == '566127.html', 'mics_etc'] = 'bolingbrook 16'
#1293
music_df.loc[music_df['id'] == '566132.html', 'date_by_hand'] = pandas.to_datetime('2016-07-06')
music_df.loc[music_df['id'] == '566132.html', 'mics_etc'] = '201'
#1305
music_df.loc[music_df['id'] == '566148.html', 'date_by_hand'] = pandas.to_datetime('2006-12-11')
music_df.loc[music_df['id'] == '566148.html', 'mics_etc'] = 'berlin_16_tr01'
#1314
music_df.loc[music_df['id'] == '566157.html', 'date_by_hand'] = pandas.to_datetime('1979-11-24')
music_df.loc[music_df['id'] == '566157.html', 'mics_etc'] = 'southampton_tr15'
#1356
music_df.loc[music_df['id'] == '566210.html', 'date_by_hand'] = pandas.to_datetime('2016-01-08')
#1365
music_df.loc[music_df['id'] == '566219.html', 'date_by_hand'] = pandas.to_datetime('1990-02-17')
music_df.loc[music_df['id'] == '566157.html', 'mics_etc'] = 'pdp_loungeax'
music_df.loc[music_df['id'] == '566157.html', 'mics_etc'] = 'd6_909_t01'
#1369
music_df.loc[music_df['id'] == '566223.html', 'date_by_hand'] = pandas.to_datetime('1988-07-09')
music_df.loc[music_df['id'] == '566157.html', 'band'] = ''
music_df.loc[music_df['id'] == '566157.html', 'mics_etc'] = '07'
#1377
music_df.loc[music_df['id'] == '566234.html', 'date_by_hand'] = pandas.to_datetime('1979-11-18')
music_df.loc[music_df['id'] == '566234.html', 'mics_etc'] = 'poole_15'
#1393
music_df.loc[music_df['id'] == '566252.html', 'date_by_hand'] = pandas.to_datetime('2016-07-09')
music_df.loc[music_df['id'] == '566252.html', 'mics_etc'] = 'royals stadium kansas city missouri'
#1403
music_df.loc[music_df['id'] == '566264.html', 'date_by_hand'] = pandas.to_datetime('2016-07-10')
music_df.loc[music_df['id'] == '566264.html', 'mics_etc'] = 't01'
#1449
music_df.loc[music_df['id'] == '566331.html', 'date_by_hand'] = pandas.to_datetime('2006-12-11')
music_df.loc[music_df['id'] == '566331.html', 'mics_etc'] = 'berlin_16_tr01'
#1683
music_df.loc[music_df['id'] == '566657.html', 'date_by_hand'] = pandas.to_datetime('2016-07-11')
music_df.loc[music_df['id'] == '566657.html', 'mics_etc'] = 'montreux (biddie)_track 17'
#1701
music_df.loc[music_df['id'] == '566680.html', 'date_by_hand'] = pandas.to_datetime('2016-07-11')
music_df.loc[music_df['id'] == '566680.html', 'mics_etc'] = '115'
#1784
music_df.loc[music_df['id'] == '566783.html', 'date_by_hand'] = pandas.to_datetime('2016-07-15')
music_df.loc[music_df['id'] == '566783.html', 'mics_etc'] = 't01'
#1956
music_df.loc[music_df['id'] == '567003.html', 'date_by_hand'] = pandas.to_datetime('2016-07-17')
music_df.loc[music_df['id'] == '567003.html', 'mics_etc'] = '01-001a_1851'
#1990
music_df.loc[music_df['id'] == '567053.html', 'date_by_hand'] = pandas.to_datetime('1981-08-20')
music_df.loc[music_df['id'] == '567053.html', 'band'] = 'ramones'
music_df.loc[music_df['id'] == '567053.html', 'mics_etc'] = '21scruzcivic'
#2053
music_df.loc[music_df['id'] == '567140.html', 'date_by_hand'] = pandas.to_datetime('2016-07-20')
music_df.loc[music_df['id'] == '567140.html', 'mics_etc'] = 'independence mo events center 01'
#2058
music_df.loc[music_df['id'] == '567146.html', 'date_by_hand'] = pandas.to_datetime('2013-04-01')
music_df.loc[music_df['id'] == '567146.html', 'band'] = 'dust bowl'
music_df.loc[music_df['id'] == '567146.html', 'mics_etc'] = '01. --paris '
#2112
music_df.loc[music_df['id'] == '567220.html', 'date_by_hand'] = pandas.to_datetime('2016-07-02')
music_df.loc[music_df['id'] == '567220.html', 'mics_etc'] = '15) where it\'s at - good times - china girl - pocket calculator - 1999 medley '
#2200
music_df.loc[music_df['id'] == '567318.html', 'date_by_hand'] = pandas.to_datetime('2011-09-02')
music_df.loc[music_df['id'] == '567318.html', 'mics_etc'] = 'birmingham _disc 01 track 04'
#2298
music_df.loc[music_df['id'] == '567438.html', 'date_by_hand'] = pandas.to_datetime('2016-07-12')
music_df.loc[music_df['id'] == '567438.html', 'mics_etc'] = '001'
#2403
music_df.loc[music_df['id'] == '567578.html', 'date_by_hand'] = pandas.to_datetime('2016-07-08')
music_df.loc[music_df['id'] == '567578.html', 'band'] = 'trembling bells'
music_df.loc[music_df['id'] == '567578.html', 'mics_etc'] = 'aud01'
#2449
music_df.loc[music_df['id'] == '567639.html', 'date_by_hand'] = pandas.to_datetime('1975-01-03')
music_df.loc[music_df['id'] == '567639.html', 'band'] = 'rrk'
music_df.loc[music_df['id'] == '567639.html', 'mics_etc'] = 'village_vanguard_u t'
#2474
music_df.loc[music_df['id'] == '567671.html', 'date_by_hand'] = pandas.to_datetime('1989-11-19')
music_df.loc[music_df['id'] == '567671.html', 'mics_etc'] = '07'
#2567 
music_df.loc[music_df['id'] == '567790.html', 'date_by_hand'] = pandas.to_datetime('2016-07-26')
music_df.loc[music_df['id'] == '567790.html', 'mics_etc'] = 'aud.mk4.petaluma.flac16 01'
#2630
music_df.loc[music_df['id'] == '567872.html', 'date_by_hand'] = pandas.to_datetime('2016-07-30')
music_df.loc[music_df['id'] == '567872.html', 'mics_etc'] = ''
#2723
music_df.loc[music_df['id'] == '126880.html', 'date_by_hand'] = pandas.to_datetime('1988-10-01')
music_df.loc[music_df['id'] == '126880.html', 'mics_etc'] = ''


print('\nStep 9a. Band, date by hand format: score = 1')
ones_df = ones_df.append(music_df.loc[(music_df['band'] == '') & (music_df['date_by_hand'].notnull())])
music_df = music_df.loc[~music_df.index.isin(ones_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 9b. Band, date by hand format: score = 1')
twos_df = twos_df.append(music_df.loc[(music_df['band'] != '') & (music_df['date_by_hand'].notnull())])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

#music_df['filename'].loc[1393]

In [ ]:
music_df['year'] = -9999
music_df['mics_etc'] = ''
#185
music_df.loc[music_df['id'] == 'e059e8d39da4b75145b1e434b0569df9c8f59367', 'year'] = 1995
music_df.loc[music_df['id'] == 'e059e8d39da4b75145b1e434b0569df9c8f59367', 'band'] = 'the ramones'
music_df.loc[music_df['id'] == 'e059e8d39da4b75145b1e434b0569df9c8f59367', 'mics_etc'] = 'd1t01.durango 95'
#219
music_df.loc[music_df['id'] == '8a7be6c05e405264de580672fbe6619a79955c43', 'year'] = 1975
music_df.loc[music_df['id'] == '8a7be6c05e405264de580672fbe6619a79955c43', 'mics_etc'] = 't01'
#224
music_df.loc[music_df['id'] == 'jg1975-xx-xx.boswell-smith.flac16', 'year'] = 1975
music_df.loc[music_df['id'] == 'jg1975-xx-xx.boswell-smith.flac16', 'mics_etc'] = 't01'
#271
music_df.loc[music_df['id'] == 'c23154fca319e57ed53f15a6a84e87a539312555', 'year'] = 1968
music_df.loc[music_df['id'] == 'c23154fca319e57ed53f15a6a84e87a539312555', 'mics_etc'] = 'los angeles,ca jonathan winters show mk1-tv-01'
#272
music_df.loc[music_df['id'] == '17a8a1fb659a7ccbc3df9d710ca2b1be9504c5c4', 'year'] = 1968
music_df.loc[music_df['id'] == '17a8a1fb659a7ccbc3df9d710ca2b1be9504c5c4', 'mics_etc'] = 'los angeles,ca jonathan winters show mk2-tv-01'
#388
music_df.loc[music_df['id'] == '587452.html', 'year'] = 1975
music_df.loc[music_df['id'] == '587452.html', 'month'] = 3
music_df.loc[music_df['id'] == '587452.html', 'mics_etc'] = 'xxt01'
#714
music_df.loc[music_df['id'] == '587815.html', 'year'] = 2016
music_df.loc[music_df['id'] == '587815.html', 'mics_etc'] = 't01'
#1223
music_df.loc[music_df['id'] == '566034.html', 'year'] = 1978
music_df.loc[music_df['id'] == '566034.html', 'mics_etc'] = 'gettysburg-tr01 entwined'
#1250
music_df.loc[music_df['id'] == '566072.html', 'year'] = 1985
music_df.loc[music_df['id'] == '566072.html', 'month'] = 10
music_df.loc[music_df['id'] == '566072.html', 'mics_etc'] = 'xxd01t01'
#1282
music_df.loc[music_df['id'] == '566119.html', 'year'] = 2016
music_df.loc[music_df['id'] == '566119.html', 'band'] = 'iggy pop'
music_df.loc[music_df['id'] == '566119.html', 'mics_etc'] = '01-isle of wight 2016.'
#1355
music_df.loc[music_df['id'] == '566208.html', 'year'] = 1972
music_df.loc[music_df['id'] == '566208.html', 'month'] = 1
music_df.loc[music_df['id'] == '566208.html', 'mics_etc'] = 'danish radio big band'
music_df.loc[music_df['id'] == '566208.html', 'mics_etc'] = 'k&atilde;&para;lner funkhaus'
#1375
music_df.loc[music_df['id'] == '566232.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566232.html', 'mics_etc'] = 'titel 01'
#1394
music_df.loc[music_df['id'] == '566253.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566232.html', 'mics_etc'] = '01 track0101'
#1460
music_df.loc[music_df['id'] == '566348.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566348.html', 'mics_etc'] = '01 all or nothing at all'
#1579
music_df.loc[music_df['id'] == '566506.html', 'year'] = 2016
music_df.loc[music_df['id'] == '566506.html', 'month'] = 7
music_df.loc[music_df['id'] == '566506.html', 'mics_etc'] = 'track01'
#1637
music_df.loc[music_df['id'] == '566581.html', 'year'] = 1998
music_df.loc[music_df['id'] == '566581.html', 'month'] = 1
music_df.loc[music_df['id'] == '566581.html', 'mics_etc'] = 'elvin'
#1759
music_df.loc[music_df['id'] == '566756.html', 'year'] = 2011
music_df.loc[music_df['id'] == '566756.html', 'mics_etc'] = '01'
#1761
music_df.loc[music_df['id'] == '566758.html', 'year'] = 1991
music_df.loc[music_df['id'] == '566758.html', 'month'] = 7
music_df.loc[music_df['id'] == '566758.html', 'mics_etc'] = 't01'
#1811
music_df.loc[music_df['id'] == '566814.html', 'year'] = 2016
music_df.loc[music_df['id'] == '566814.html', 'mics_etc'] = 'montreux_01'
#2038
music_df.loc[music_df['id'] == '567117.html', 'year'] = 1986
music_df.loc[music_df['id'] == '567117.html', 'month'] = 6
music_df.loc[music_df['id'] == '567117.html', 'mics_etc'] = 'tigers-t01'
#2087
music_df.loc[music_df['id'] == '567186.html', 'year'] = 1986
music_df.loc[music_df['id'] == '567186.html', 'month'] = 6
music_df.loc[music_df['id'] == '567186.html', 'mics_etc'] = 'paradise-t01'
#2114
music_df.loc[music_df['id'] == '567222.html', 'year'] = 2011
music_df.loc[music_df['id'] == '567222.html', 'month'] = 2
music_df.loc[music_df['id'] == '567222.html', 'mics_etc'] = 'heroes'
#2156
music_df.loc[music_df['id'] == '567271.html', 'year'] = 2016
music_df.loc[music_df['id'] == '567271.html', 'band'] = 'editors'
music_df.loc[music_df['id'] == '567271.html', 'mics_etc'] = '01 glastonbury'
#2204
music_df.loc[music_df['id'] == '567323.html', 'year'] = 1983
music_df.loc[music_df['id'] == '567323.html', 'band'] = 'peter gabriel'
music_df.loc[music_df['id'] == '567323.html', 'mics_etc'] = '02_paris 83 -cd2'
#2211
music_df.loc[music_df['id'] == '567333.html', 'year'] = 2007
music_df.loc[music_df['id'] == '567333.html', 'month'] = 7
music_df.loc[music_df['id'] == '567333.html', 'band'] = ' silver machine and waiting for the man'
#2230
music_df.loc[music_df['id'] == '567358.html', 'year'] = 2014
music_df.loc[music_df['id'] == '567358.html', 'mics_etc'] = ''
#2264
music_df.loc[music_df['id'] == '567394.html', 'year'] = 1986
music_df.loc[music_df['id'] == '567394.html', 'month'] = 5
music_df.loc[music_df['id'] == '567394.html', 'mics_etc'] = 'fm(1st-gen)t04'
#2328
music_df.loc[music_df['id'] == '567478.html', 'year'] = 2006
music_df.loc[music_df['id'] == '567478.html', 'mics_etc'] = '01 these are the god days'
#2335
music_df.loc[music_df['id'] == '567487.html', 'year'] = 2014
music_df.loc[music_df['id'] == '567487.html', 'month'] = 8
music_df.loc[music_df['id'] == '567487.html', 'mics_etc'] = '01 flying over the odra valley'
#2364
music_df.loc[music_df['id'] == '567526.html', 'year'] = 2013
music_df.loc[music_df['id'] == '567526.html', 'year'] = 10
music_df.loc[music_df['id'] == '567526.html', 'mics_etc'] = '03 sly eyes 13-10'
#2484
music_df.loc[music_df['id'] == '567683.html', 'year'] = 2017
music_df.loc[music_df['id'] == '567683.html', 'mics_etc'] = '01 miami (seen the lights go out on broadway)'
#2486 
music_df.loc[music_df['id'] == '567686.html', 'year'] = 1968
music_df.loc[music_df['id'] == '567686.html', 'mics_etc'] = 'los angeles,ca jonathan winters show mk1-tv-01'
#2487
music_df.loc[music_df['id'] == '567687.html', 'year'] = 1968
music_df.loc[music_df['id'] == '567687.html', 'mics_etc'] = 'los angeles,ca jonathan winters show mk2-tv-01'
#2580
music_df.loc[music_df['id'] == '567810.html', 'year'] = 2016
music_df.loc[music_df['id'] == '567810.html', 'band'] = 'matthew shipp'
music_df.loc[music_df['id'] == '567810.html', 'mics_etc'] = '01 taktlos'
#2828
music_df.loc[music_df['id'] == '127033.html', 'year'] = 1991
music_df.loc[music_df['id'] == '127033.html', 'mics_etc'] = 'humble pie reunion 44khz16b t01'
#2851
music_df.loc[music_df['id'] == '127062.html', 'year'] = 2017
music_df.loc[music_df['id'] == '127062.html', 'mics_etc'] = '02. miami (seen the lights go out on broadway)'

print('\nStep 10a. No band, year by hand format: score = 1')
ones_df = ones_df.append(music_df.loc[(music_df['band'] == '') & (music_df['year'] != -9999)])
music_df = music_df.loc[~music_df.index.isin(ones_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 10b. Band, year by hand format: score = 1')
twos_df = twos_df.append(music_df.loc[(music_df['band'] != '') & (music_df['year'] != -9999)])
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))


In [ ]:
music_df['nodate'] = False
#87
music_df.loc[music_df['id'] == 'ddbaea0b694a4c0deaa8f75156446b8e7adea2d6', 'nodate'] = True
music_df.loc[music_df['id'] == 'ddbaea0b694a4c0deaa8f75156446b8e7adea2d6', 'mics_etc'] = '201'
#143
music_df.loc[music_df['id'] == 'f808ec4b46eb140fca6b9b73f6cbff76ee0e6641', 'nodate'] = True
music_df.loc[music_df['id'] == 'f808ec4b46eb140fca6b9b73f6cbff76ee0e6641', 'band'] = 'neil young'
music_df.loc[music_df['id'] == 'f808ec4b46eb140fca6b9b73f6cbff76ee0e6641', 'mics_etc'] = 'd1t01 my my hey hey (out of the blue)'
#177
music_df.loc[music_df['id'] == '888ae778faf1d5ed69020113bf9f40806478f4ac', 'nodate'] = True
music_df.loc[music_df['id'] == '888ae778faf1d5ed69020113bf9f40806478f4ac', 'mics_etc'] = 't02.jack straw'
#183
music_df.loc[music_df['id'] == '52677374ccbd20cc35dbce25ac5b7018e9a8af0a', 'nodate'] = True
music_df.loc[music_df['id'] == '52677374ccbd20cc35dbce25ac5b7018e9a8af0a', 'band'] = ''
music_df.loc[music_df['id'] == '52677374ccbd20cc35dbce25ac5b7018e9a8af0a', 'mics_etc'] = 'd1t01'
#186
music_df.loc[music_df['id'] == 'edf25ec0ac4fd08874c7627bc527420091ef1747', 'nodate'] = True
music_df.loc[music_df['id'] == 'edf25ec0ac4fd08874c7627bc527420091ef1747', 'mics_etc'] = 'sl01_intro'
#192
music_df.loc[music_df['id'] == 'ff3fe93793664814e48139fbd51022e73347ec86', 'nodate'] = True
music_df.loc[music_df['id'] == 'ff3fe93793664814e48139fbd51022e73347ec86', 'band'] = 'the ramones'
music_df.loc[music_df['id'] == 'ff3fe93793664814e48139fbd51022e73347ec86', 'mics_etc'] = 'rockaway beach'
#233
music_df.loc[music_df['id'] == '5a3b55bc30d8dabb1023d442759c34dd70edebb1', 'nodate'] = True
music_df.loc[music_df['id'] == '5a3b55bc30d8dabb1023d442759c34dd70edebb1', 'band'] = 'br'
music_df.loc[music_df['id'] == '5a3b55bc30d8dabb1023d442759c34dd70edebb1', 'mics_etc'] = '01'
#234
music_df.loc[music_df['id'] == 'ebb2e1757bdb2a01716acefca2697fd718d9a8c1', 'nodate'] = True
music_df.loc[music_df['id'] == 'ebb2e1757bdb2a01716acefca2697fd718d9a8c1', 'band'] = 'rush'
music_df.loc[music_df['id'] == 'ebb2e1757bdb2a01716acefca2697fd718d9a8c1', 'mics_etc'] = '1t01.-a passage to bangkok'
#239
music_df.loc[music_df['id'] == '1c5dc8fe10c17dcf42a786c638d6107939b81ef1', 'nodate'] = True
music_df.loc[music_df['id'] == '1c5dc8fe10c17dcf42a786c638d6107939b81ef1', 'mics_etc'] = '01 455 rocket'
print('Up to 500...')
#614
music_df.loc[music_df['id'] == '587704.html', 'nodate'] = True
music_df.loc[music_df['id'] == '587704.html', 'band'] = 'matrix'
music_df.loc[music_df['id'] == '587704.html', 'mics_etc'] = '01'
#651
music_df.loc[music_df['id'] == '587748.html', 'nodate'] = True
music_df.loc[music_df['id'] == '587748.html', 'band'] = 'matrix'
music_df.loc[music_df['id'] == '587748.html', 'mics_etc'] = '01'
#736
music_df.loc[music_df['id'] == '587838.html', 'nodate'] = True
music_df.loc[music_df['id'] == '587838.html', 'band'] = 't.flyer'
music_df.loc[music_df['id'] == '587838.html', 'mics_etc'] = 'disc01,track01'
#840
music_df.loc[music_df['id'] == '565503.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565503.html', 'mics_etc'] = 'le cure en montreal - 01 out of this world'
#870
music_df.loc[music_df['id'] == '565549.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565549.html', 'mics_etc'] = '1track01'
#879
music_df.loc[music_df['id'] == '565558.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565558.html', 'band'] = ''
music_df.loc[music_df['id'] == '565558.html', 'mics_etc'] = '1-01'
#902
music_df.loc[music_df['id'] == '565597.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565597.html', 'mics_etc'] = 'd1t01'
#910
music_df.loc[music_df['id'] == '565606.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565606.html', 'mics_etc'] = 'd1t01'
#919
music_df.loc[music_df['id'] == '565616.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565616.html', 'mics_etc'] = '210 - rosalita'
#939
music_df.loc[music_df['id'] == '565641.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565641.html', 'band'] = ''
music_df.loc[music_df['id'] == '565641.html', 'mics_etc'] = 'seattle2track01'
#948
music_df.loc[music_df['id'] == '565654.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565654.html', 'mics_etc'] = '01'
#950
music_df.loc[music_df['id'] == '565657.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565657.html', 'mics_etc'] = '205 atom heart mother'
#963
music_df.loc[music_df['id'] == '565675.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565675.html', 'mics_etc'] = '001_01'
#964
music_df.loc[music_df['id'] == '565676.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565676.html', 'mics_etc'] = 'sweet liberties'
music_df.loc[music_df['id'] == '565676.html', 'mics_etc'] = 'd101'
#970
music_df.loc[music_df['id'] == '565684.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565684.html', 'mics_etc'] = '08'
#977
music_df.loc[music_df['id'] == '565694.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565694.html', 'mics_etc'] = '01'
#982
music_df.loc[music_df['id'] == '565702.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565702.html', 'mics_etc'] = '1t01'
#993
music_df.loc[music_df['id'] == '565715.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565715.html', 'mics_etc'] = '101 the motel'
#994
music_df.loc[music_df['id'] == '565716.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565716.html', 'mics_etc'] = '209 light of day'
print('Up to 1000...')
#1007
music_df.loc[music_df['id'] == '565738.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565738.html', 'band'] = ''
music_df.loc[music_df['id'] == '565738.html', 'mics_etc'] = '209 light of day'
#1013
music_df.loc[music_df['id'] == '565746.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565746.html', 'mics_etc'] = 'on fire'
#1027
music_df.loc[music_df['id'] == '565763.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565763.html', 'band'] = ''
music_df.loc[music_df['id'] == '565763.html', 'mics_etc'] = 't01 - intro - land of hope &amp; glory - over the rainbow'
#1028
music_df.loc[music_df['id'] == '565764.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565764.html', 'mics_etc'] = '01'
#1030
music_df.loc[music_df['id'] == '565771.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565771.html', 'band'] = ''
music_df.loc[music_df['id'] == '565771.html', 'mics_etc'] = 'd1t01'
#1056
music_df.loc[music_df['id'] == '565804.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565804.html', 'mics_etc'] = '101 intro'
#1069
music_df.loc[music_df['id'] == '565826.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565826.html', 'band'] = ''
music_df.loc[music_df['id'] == '565826.html', 'mics_etc'] = 'r09_0004_r8b01'
#1082
music_df.loc[music_df['id'] == '565844.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565844.html', 'mics_etc'] = '01 hold on i\'m coming'
#1094
music_df.loc[music_df['id'] == '565863.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565863.html', 'band'] = ''
music_df.loc[music_df['id'] == '565863.html', 'mics_etc'] = 'd1t01'
#1119
music_df.loc[music_df['id'] == '565896.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565896.html', 'mics_etc'] = '01'
#1124
music_df.loc[music_df['id'] == '565901.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565901.html', 'band'] = 'whitford'
music_df.loc[music_df['id'] == '565901.html', 'mics_etc'] = 'st01'
#1132
music_df.loc[music_df['id'] == '565911.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565911.html', 'mics_etc'] = 'st01'
#1135
music_df.loc[music_df['id'] == '565915.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565915.html', 'mics_etc'] = 'd1t01'
#1138
music_df.loc[music_df['id'] == '565922.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565922.html', 'mics_etc'] = '01 kyrie aus der messe in h-moll 01-36'
#1145
music_df.loc[music_df['id'] == '565929.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565929.html', 'mics_etc'] = '01 prometheus 06-31'
#1151
music_df.loc[music_df['id'] == '565938.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565938.html', 'mics_etc'] = '07'
#1172
music_df.loc[music_df['id'] == '565966.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565966.html', 'mics_etc'] = '01 tribute to keith jarrett 11-39'
#1181
music_df.loc[music_df['id'] == '565981.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565981.html', 'mics_etc'] = '01 scollin\' 08-29'
#1194
music_df.loc[music_df['id'] == '565997.html', 'nodate'] = True
music_df.loc[music_df['id'] == '565997.html', 'band'] = ''
music_df.loc[music_df['id'] == '565997.html', 'mics_etc'] = 'a01'
#1200
music_df.loc[music_df['id'] == '566005.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566005.html', 'mics_etc'] = '101 mr. tambourine man'
#1213
music_df.loc[music_df['id'] == '566020.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566020.html', 'mics_etc'] = 'd2t06 - biko'
#1230
music_df.loc[music_df['id'] == '566046.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566046.html', 'mics_etc'] = '101 - [venue announcements]'
#1233
music_df.loc[music_df['id'] == '566050.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566050.html', 'band'] = 'black sabbath'
music_df.loc[music_df['id'] == '566050.html', 'mics_etc'] = 'a01'
#1316
music_df.loc[music_df['id'] == '566159.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566159.html', 'mics_etc'] = '01 shady grove'
#1326
music_df.loc[music_df['id'] == '566170.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566170.html', 'mics_etc'] = '01 1'
#1328
music_df.loc[music_df['id'] == '566175.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566175.html', 'mics_etc'] = '01'
#1331
music_df.loc[music_df['id'] == '566178.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566178.html', 'mics_etc'] = '00'
#1334
music_df.loc[music_df['id'] == '566181.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566181.html', 'band'] = ''
music_df.loc[music_df['id'] == '566181.html', 'mics_etc'] = 't01 after the goldrush'
#1354
music_df.loc[music_df['id'] == '566207.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566207.html', 'band'] = ''
music_df.loc[music_df['id'] == '566207.html', 'mics_etc'] = 'd1t01'
#1361
music_df.loc[music_df['id'] == '566215.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566215.html', 'mics_etc'] = '01 little black dress'
#1375
music_df.loc[music_df['id'] == '566232.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566232.html', 'band'] = ''
music_df.loc[music_df['id'] == '566232.html', 'mics_etc'] = 'titel 01'
#1394
music_df.loc[music_df['id'] == '566253.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566253.html', 'mics_etc'] = '01 track0101'
#1404
music_df.loc[music_df['id'] == '566266.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566266.html', 'mics_etc'] = '01 ode to satie 11-24'
#1416
music_df.loc[music_df['id'] == '566280.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566280.html', 'band'] = 'thunderstone'
music_df.loc[music_df['id'] == '566280.html', 'mics_etc'] = 'soundcheck'
#1439
music_df.loc[music_df['id'] == '566317.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566317.html', 'mics_etc'] = '1-01 rainy day women #12 &amp; 35'
#1441
music_df.loc[music_df['id'] == '566320.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566320.html', 'mics_etc'] = '01'
#1460
music_df.loc[music_df['id'] == '566348.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566348.html', 'mics_etc'] = '01 all or nothing at all'
#1476
music_df.loc[music_df['id'] == '566369.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566369.html', 'mics_etc'] = '208-light of day'
#1483
music_df.loc[music_df['id'] == '566377.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566377.html', 'mics_etc'] = 'marque 01'
#1491
music_df.loc[music_df['id'] == '566392.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566392.html', 'mics_etc'] = 'das rheingold part 1'
print('Up to 1500...')
#1512
music_df.loc[music_df['id'] == '566417.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566417.html', 'mics_etc'] = '01 interiors 13-38'
#1516
music_df.loc[music_df['id'] == '566421.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566421.html', 'mics_etc'] = '04'
#1530
music_df.loc[music_df['id'] == '566436.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566436.html', 'mics_etc'] = '204 mary\'s place'
#1533
music_df.loc[music_df['id'] == '566439.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566439.html', 'mics_etc'] = 'hw'
music_df.loc[music_df['id'] == '566439.html', 'mics_etc'] = 'toads place matrix _01'
#1535
music_df.loc[music_df['id'] == '566445.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566445.html', 'mics_etc'] = '101 - 3 stooges intro'
#1539
music_df.loc[music_df['id'] == '566451.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566451.html', 'mics_etc'] = '01 hot tamale baby'
#1550
music_df.loc[music_df['id'] == '566465.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566465.html', 'mics_etc'] = '01 - die walkure act 1'
#1562
music_df.loc[music_df['id'] == '566485.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566485.html', 'mics_etc'] = '01 dori theme 06-35'
#1567
music_df.loc[music_df['id'] == '566491.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566491.html', 'mics_etc'] = 't01'
#1568
music_df.loc[music_df['id'] == '566492.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566492.html', 'mics_etc'] = '01 unknown'
#1595
music_df.loc[music_df['id'] == '566525.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566525.html', 'mics_etc'] = '02 - 01-stratovarius'
#1606
music_df.loc[music_df['id'] == '566539.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566539.html', 'mics_etc'] = ' 01 - siegfried act 1'
#1622
music_df.loc[music_df['id'] == '566560.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566560.html', 'mics_etc'] = '22'
#1624
music_df.loc[music_df['id'] == '566563.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566568.html', 'mics_etc'] = '1-04 give me what i want and give it'
#1627
music_df.loc[music_df['id'] == '566568.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566568.html', 'mics_etc'] = 'd1track01'
#1631
music_df.loc[music_df['id'] == '566572.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566572.html', 'mics_etc'] = '06'
#1636
music_df.loc[music_df['id'] == '566580.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566580.html', 'mics_etc'] = '101 save it for me'
#1641
music_df.loc[music_df['id'] == '566594.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566594.html', 'mics_etc'] = 'kv01'
#1643
music_df.loc[music_df['id'] == '566598.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566598.html', 'band'] = ''
music_df.loc[music_df['id'] == '566598.html', 'mics_etc'] = 'd01'
#1644
music_df.loc[music_df['id'] == '566604.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566604.html', 'band'] = ''
music_df.loc[music_df['id'] == '566604.html', 'mics_etc'] = 'n01'
#1645
music_df.loc[music_df['id'] == '566607.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566607.html', 'band'] = ''
music_df.loc[music_df['id'] == '566607.html', 'mics_etc'] = 'tt01'
#1656
music_df.loc[music_df['id'] == '566621.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566621.html', 'mics_etc'] = '01 - g&atilde;&para;tterd&atilde;&curren;mmerung act 1 part 1'
#1666
music_df.loc[music_df['id'] == '566636.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566636.html', 'band'] = ''
music_df.loc[music_df['id'] == '566636.html', 'mics_etc'] = 'd1t01 little liza jane 08-51'
#1680
music_df.loc[music_df['id'] == '566654.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566654.html', 'mics_etc'] = '01'
#1692
music_df.loc[music_df['id'] == '566669.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566669.html', 'mics_etc'] = '302 - rosalita'
#1698
music_df.loc[music_df['id'] == '566676.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566676.html', 'mics_etc'] = '101 5 am'
#1720
music_df.loc[music_df['id'] == '566707.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566707.html', 'mics_etc'] = '02'
#1726
music_df.loc[music_df['id'] == '566716.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566716.html', 'band'] = ''
music_df.loc[music_df['id'] == '566716.html', 'mics_etc'] = 'd1t01-highballwiththedevil'
#1728
music_df.loc[music_df['id'] == '566722.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566722.html', 'mics_etc'] = '01 - unknown'
#1729
music_df.loc[music_df['id'] == '566723.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566723.html', 'band'] = 'earthrise'
music_df.loc[music_df['id'] == '566723.html', 'mics_etc'] = 'first night-rhayder-uneven song'
#1732
music_df.loc[music_df['id'] == '566727.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566727.html', 'band'] = 'buben'
music_df.loc[music_df['id'] == '566727.html', 'mics_etc'] = 'cin 06'
#1733
music_df.loc[music_df['id'] == '566728.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566728.html', 'mics_etc'] = '23'
#1746
music_df.loc[music_df['id'] == '566743.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566743.html', 'mics_etc'] = '02'
#1749
music_df.loc[music_df['id'] == '566746.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566746.html', 'band'] = ''
music_df.loc[music_df['id'] == '566746.html', 'mics_etc'] = 'd101-intro'
#1756
music_df.loc[music_df['id'] == '566753.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566753.html', 'mics_etc'] = '13'
#1757
music_df.loc[music_df['id'] == '566754.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566754.html', 'mics_etc'] = '01'
#1760
music_df.loc[music_df['id'] == '566757.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566757.html', 'band'] = ''
music_df.loc[music_df['id'] == '566757.html', 'mics_etc'] = 'd1 track01'
#1764
music_df.loc[music_df['id'] == '566762.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566762.html', 'band'] = ''
music_df.loc[music_df['id'] == '566762.html', 'mics_etc'] = 'd1 track01'
#1772
music_df.loc[music_df['id'] == '566770.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566770.html', 'mics_etc'] = '01 room 335'
#1775
music_df.loc[music_df['id'] == '566774.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566774.html', 'mics_etc'] = '01 room 335'
#1779
music_df.loc[music_df['id'] == '566778.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566778.html', 'mics_etc'] = '01 room 335'
#1800
music_df.loc[music_df['id'] == '566802.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566802.html', 'mics_etc'] = '01 strange as it seems'
#1812
music_df.loc[music_df['id'] == '566815.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566815.html', 'mics_etc'] = '01 intro'
#1816
music_df.loc[music_df['id'] == '566820.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566820.html', 'band'] = 'redford'
music_df.loc[music_df['id'] == '566820.html', 'mics_etc'] = '01 (for yia-yia &amp; pappou)'
#1825
music_df.loc[music_df['id'] == '566834.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566834.html', 'mics_etc'] = '01 strikes twice'
#1826
music_df.loc[music_df['id'] == '566836.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566836.html', 'mics_etc'] = '01 nite crawler'
#1830
music_df.loc[music_df['id'] == '566844.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566844.html', 'mics_etc'] = '001'
#1831
music_df.loc[music_df['id'] == '566845.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566845.html', 'band'] = ''
music_df.loc[music_df['id'] == '566845.html', 'mics_etc'] = 'd1 01 after the goldrush'
#1834
music_df.loc[music_df['id'] == '566849.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566849.html', 'band'] = ''
music_df.loc[music_df['id'] == '566849.html', 'mics_etc'] = 't01 after the goldrush'
#1835
music_df.loc[music_df['id'] == '566850.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566850.html', 'band'] = ''
music_df.loc[music_df['id'] == '566850.html', 'mics_etc'] = '01'
#1847
music_df.loc[music_df['id'] == '566865.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566865.html', 'mics_etc'] = '01 friday night shuffle'
#1848
music_df.loc[music_df['id'] == '566867.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566867.html', 'mics_etc'] = '302 - comfortably numb (cut)'
#1867
music_df.loc[music_df['id'] == '566894.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566894.html', 'mics_etc'] = '209-light of day'
#1869
music_df.loc[music_df['id'] == '566896.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566896.html', 'mics_etc'] = 'final01'
#1875
music_df.loc[music_df['id'] == '566903.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566903.html', 'band'] = 'dixie chicks'
music_df.loc[music_df['id'] == '566903.html', 'mics_etc'] = 'mmxvi (00) prince - lets go crazy (end)'
#1877
music_df.loc[music_df['id'] == '566905.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566905.html', 'mics_etc'] = '01 unknown'
#1895
music_df.loc[music_df['id'] == '566928.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566928.html', 'mics_etc'] = '01 instrumental'
#1897 
music_df.loc[music_df['id'] == '566930.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566930.html', 'mics_etc'] = '01 interview with carla bley and steve swallow.mp3'
#1899
music_df.loc[music_df['id'] == '566932.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566932.html', 'mics_etc'] = '01 interview with ches smith'
#1924
music_df.loc[music_df['id'] == '566961.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566961.html', 'mics_etc'] = 'd101'
#1929
music_df.loc[music_df['id'] == '566967.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566967.html', 'mics_etc'] = '1 16 bit01'
#1932
music_df.loc[music_df['id'] == '566972.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566972.html', 'mics_etc'] = '101 mojito moon (michaela\'s song)'
#1934
music_df.loc[music_df['id'] == '566976.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566976.html', 'mics_etc'] = 'd3t04'
#1938
music_df.loc[music_df['id'] == '566982.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566982.html', 'mics_etc'] = '01 minor mood 11-59'
#1946
music_df.loc[music_df['id'] == '566991.html', 'nodate'] = True
music_df.loc[music_df['id'] == '566991.html', 'mics_etc'] = '2 4401'
#1953
music_df.loc[music_df['id'] == '567000.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567000.html', 'band'] = ''
music_df.loc[music_df['id'] == '567000.html', 'mics_etc'] = 't01-hell on wheels'
#1954
music_df.loc[music_df['id'] == '567001.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567001.html', 'band'] = ''
music_df.loc[music_df['id'] == '567001.html', 'mics_etc'] = 'd02t05 - pay me my money down'
#1967
music_df.loc[music_df['id'] == '567022.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567022.html', 'mics_etc'] = '01-beck\'s bolero - rice pudding'
#1970
music_df.loc[music_df['id'] == '567025.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567025.html', 'mics_etc'] = '01-beck\'s bolero - rice pudding'
#1985
music_df.loc[music_df['id'] == '567048.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567048.html', 'mics_etc'] = '01 nacht 08-38'
#1992
music_df.loc[music_df['id'] == '567055.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567055.html', 'mics_etc'] = '101 something in the way she moves'
#1998
music_df.loc[music_df['id'] == '567062.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567062.html', 'mics_etc'] = '01'
print('Up to 2000...')
#2002
music_df.loc[music_df['id'] == '567068.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567068.html', 'mics_etc'] = '01'
#2017
music_df.loc[music_df['id'] == '567087.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567087.html', 'band'] = ''
music_df.loc[music_df['id'] == '567087.html', 'mics_etc'] = 'a01'
#2018
music_df.loc[music_df['id'] == '567088.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567088.html', 'mics_etc'] = '01-beck\'s bolero - rice pudding'
#2052
music_df.loc[music_df['id'] == '567139.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567139.html', 'mics_etc'] = '01'
#2054
music_df.loc[music_df['id'] == '567141.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567141.html', 'band'] = ''
music_df.loc[music_df['id'] == '567141.html', 'mics_etc'] = 't01 after the goldrush'
#2055
music_df.loc[music_df['id'] == '567142.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567142.html', 'mics_etc'] = '0101 dirty rain'
#2061
music_df.loc[music_df['id'] == '567150.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567150.html', 'mics_etc'] = '01-beck\'s bolero - rice pudding'
#2062
music_df.loc[music_df['id'] == '567152.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567152.html', 'band'] = ''
music_df.loc[music_df['id'] == '567152.html', 'mics_etc'] = 't01 after the goldrush'
#2063
music_df.loc[music_df['id'] == '567153.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567153.html', 'band'] = ''
music_df.loc[music_df['id'] == '567153.html', 'mics_etc'] = 'a1 what you don\'t know'
#2068
music_df.loc[music_df['id'] == '567161.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567161.html', 'band'] = ''
music_df.loc[music_df['id'] == '567161.html', 'mics_etc'] = 'cue 01'
#2069
music_df.loc[music_df['id'] == '567162.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567162.html', 'mics_etc'] = '1-01'
#2073
music_df.loc[music_df['id'] == '567167.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567167.html', 'mics_etc'] = '02'
#2091
music_df.loc[music_df['id'] == '567192.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567192.html', 'band'] = 'led zeppelin'
music_df.loc[music_df['id'] == '567192.html', 'mics_etc'] = '01 - black dog'
#2095
music_df.loc[music_df['id'] == '567198.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567198.html', 'band'] = ''
music_df.loc[music_df['id'] == '567198.html', 'mics_etc'] = 'a1 stay hungry'
#2098
music_df.loc[music_df['id'] == '567201.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567201.html', 'mics_etc'] = '01-beck\'s bolero - rice pudding'
#2099
music_df.loc[music_df['id'] == '567202.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567202.html', 'mics_etc'] = 'foxboro 201'
#2104
music_df.loc[music_df['id'] == '567207.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567207.html', 'mics_etc'] = '01'
#2121
music_df.loc[music_df['id'] == '567230.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567230.html', 'band'] = ''
music_df.loc[music_df['id'] == '567230.html', 'mics_etc'] = 'd2t04 open all night'
#2123
music_df.loc[music_df['id'] == '567233.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567233.html', 'band'] = 'led zeppelin'
music_df.loc[music_df['id'] == '567233.html', 'mics_etc'] = '101 - introduction by goro itoi'
#2124
music_df.loc[music_df['id'] == '567234.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567234.html', 'band'] = 'gnr'
music_df.loc[music_df['id'] == '567234.html', 'mics_etc'] = 'sfaud01'
#2133
music_df.loc[music_df['id'] == '567244.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567244.html', 'mics_etc'] = '101 poem'
#2146
music_df.loc[music_df['id'] == '567259.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567259.html', 'mics_etc'] = '101 opening s.e.'
#2147
music_df.loc[music_df['id'] == '567260.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567260.html', 'mics_etc'] = '101 opening s.e.'
#2160
music_df.loc[music_df['id'] == '567275.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567275.html', 'band'] = 'led zeppelin'
music_df.loc[music_df['id'] == '567275.html', 'band'] = '201 introduction'
#2168
music_df.loc[music_df['id'] == '567285.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567285.html', 'mics_etc'] = '01-beck\'s bolero - rice pudding'
#2174
music_df.loc[music_df['id'] == '567291.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567291.html', 'band'] = ''
music_df.loc[music_df['id'] == '567291.html', 'mics_etc'] = 'dt-000225_101'
#2177
music_df.loc[music_df['id'] == '567294.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567294.html', 'band'] = 'thin lizzy'
music_df.loc[music_df['id'] == '567294.html', 'mics_etc'] = '01 - intro'
#2180
music_df.loc[music_df['id'] == '567297.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567297.html', 'mics_etc'] = '101 half the truth'
#2187
music_df.loc[music_df['id'] == '567305.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567305.html', 'band'] = ''
music_df.loc[music_df['id'] == '567305.html', 'mics_etc'] = 't01'
#2190
music_df.loc[music_df['id'] == '567308.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567308.html', 'band'] = '09 - auto mowdownspace girl blues'
#2191
music_df.loc[music_df['id'] == '567309.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567309.html', 'mics_etc'] = '101'
#2201
music_df.loc[music_df['id'] == '567319.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567319.html', 'band'] = ''
music_df.loc[music_df['id'] == '567319.html', 'mics_etc'] = 'd1 01 after the gold rush'
#2203
music_df.loc[music_df['id'] == '567322.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567322.html', 'mics_etc'] = '02. 12-8 theme-esther\'s nose job'
#2209
music_df.loc[music_df['id'] == '567331.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567331.html', 'mics_etc'] = '18 - chemical warfare'
#2225
music_df.loc[music_df['id'] == '567349.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567349.html', 'mics_etc'] = '01 - nine south'
#2233
music_df.loc[music_df['id'] == '567361.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567361.html', 'mics_etc'] = 'coral creek'
music_df.loc[music_df['id'] == '567361.html', 'mics_etc'] = 'aud01'
#2236
music_df.loc[music_df['id'] == '567364.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567364.html', 'mics_etc'] = ''
music_df.loc[music_df['id'] == '567364.html', 'mics_etc'] = 'r09_0002_r8b'
#2238
music_df.loc[music_df['id'] == '567366.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567366.html', 'mics_etc'] = '01'
#2245
music_df.loc[music_df['id'] == '567373.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567373.html', 'mics_etc'] = '209 - smoke on the water'
#2246
music_df.loc[music_df['id'] == '567374.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567374.html', 'mics_etc'] = '206 - space truckin\' ~ woman from tokyo ~ paint it black'
#2248
music_df.loc[music_df['id'] == '567376.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567376.html', ''] = ''
music_df.loc[music_df['id'] == '567376.html', 'mics_etc'] = 'd2 bottom 81775_04'
#2253
music_df.loc[music_df['id'] == '567381.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567381.html', 'mics_etc'] = '01-fixed'
#2268
music_df.loc[music_df['id'] == '567400.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567400.html', 'mics_etc'] = '101 intro'
#2272
music_df.loc[music_df['id'] == '567404.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567404.html', 'mics_etc'] = '101'
#2279
music_df.loc[music_df['id'] == '567414.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567414.html', 'mics_etc'] = '01 shady grove'
#2286
music_df.loc[music_df['id'] == '567423.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567423.html', 'mics_etc'] = '01 - sunday at the hillcrest'
#2292
music_df.loc[music_df['id'] == '567429.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567429.html', 'mics_etc'] = '205 - smoke on the water'
#2293
music_df.loc[music_df['id'] == '567431.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567431.html', 'mics_etc'] = '01 the three marias 11-37'
#2309
music_df.loc[music_df['id'] == '567451.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567451.html', 'mics_etc'] = '211 dream baby dream'
#2317
music_df.loc[music_df['id'] == '567464.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567464.html', 'band'] = 'cab calloway'
music_df.loc[music_df['id'] == '567464.html', 'mics_etc'] = '01 - you\'re nobody \'til somebody loves you'
#2329
music_df.loc[music_df['id'] == '567479.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567479.html', 'mics_etc'] = '01 unknown title 11-38'
#2339
music_df.loc[music_df['id'] == '567496.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567496.html', 'band'] = 'roger mcguinn'
music_df.loc[music_df['id'] == '567496.html', 'mics_etc'] = '101 - unknown'
#2341
music_df.loc[music_df['id'] == '567499.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567499.html', 'band'] = ''
music_df.loc[music_df['id'] == '567499.html', 'mics_etc'] = 't01 5 am'
#2344
music_df.loc[music_df['id'] == '567503.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567503.html', 'band'] = 'manfred manns earthband'
music_df.loc[music_df['id'] == '567503.html', 'mics_etc'] = 'captain bobby stout'
#2345
music_df.loc[music_df['id'] == '567504.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567504.html', 'band'] = ''
music_df.loc[music_df['id'] == '567504.html', 'mics_etc'] = 't01 after the gold rush 24 bit'
#2346
music_df.loc[music_df['id'] == '567505.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567505.html', 'mics_etc'] = '02'
#2349
music_df.loc[music_df['id'] == '567509.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567509.html', 'mics_etc'] = '01 love ship 06-44'
#2360
music_df.loc[music_df['id'] == '567521.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567521.html', 'band'] = ''
music_df.loc[music_df['id'] == '567521.html', 'mics_etc'] = 't01 after the gold rush 24 bit'
#2363
music_df.loc[music_df['id'] == '567524.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567524.html', 'band'] = ''
music_df.loc[music_df['id'] == '567524.html', 'mics_etc'] = 't01 after the gold rush 24 bit'
#2365
music_df.loc[music_df['id'] == '567527.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567527.html', 'band'] = ''
music_df.loc[music_df['id'] == '567527.html', 'mics_etc'] = 't01 after the gold rush'
#2395
music_df.loc[music_df['id'] == '567570.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567570.html', 'band'] = ''
music_df.loc[music_df['id'] == '567570.html', 'mics_etc'] = 'd1t01 intro'
#2412
music_df.loc[music_df['id'] == '567590.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567590.html', 'band'] = ''
music_df.loc[music_df['id'] == '567590.html', 'mics_etc'] = 't01'
#2416
music_df.loc[music_df['id'] == '567598.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567598.html', 'mics_etc'] = '101 make me smile'
#2418
music_df.loc[music_df['id'] == '567600.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567600.html', 'band'] = ''
music_df.loc[music_df['id'] == '567600.html', 'mics_etc'] = 't01 war of kings'
#2427
music_df.loc[music_df['id'] == '567613.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567613.html', 'band'] = ''
music_df.loc[music_df['id'] == '567613.html', 'mics_etc'] = 't01 intro - jailbreak'
#2442
music_df.loc[music_df['id'] == '567632.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567632.html', 'band'] = ''
music_df.loc[music_df['id'] == '567632.html', 'mics_etc'] = 'd1t01'
#2444
music_df.loc[music_df['id'] == '567634.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567634.html', 'band'] = 'earthrise'
music_df.loc[music_df['id'] == '567634.html', 'mics_etc'] = '01 earthrise-first night-rhayder-uneven song'
#2476
music_df.loc[music_df['id'] == '567674.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567674.html', 'mics_etc'] = '01 earthrise'
#2482
music_df.loc[music_df['id'] == '567681.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567681.html', 'mics_etc'] = '01'
#2492
music_df.loc[music_df['id'] == '567692.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567692.html', 'mics_etc'] = '01 - earthrise'
#2494
music_df.loc[music_df['id'] == '567694.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567694.html', 'band'] = ''
music_df.loc[music_df['id'] == '567694.html', 'mics_etc'] = 'd1 01 after the gold rush'
#2495
music_df.loc[music_df['id'] == '567695.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567695.html', 'mics_etc'] = '01 - medley + the three marias'
print('Up to 2500...')
#2500
music_df.loc[music_df['id'] == '567700.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567700.html', 'mics_etc'] = '001'
#2504
music_df.loc[music_df['id'] == '567704.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567704.html', 'band'] = ''
music_df.loc[music_df['id'] == '567704.html', 'mics_etc'] = 'd2t07 we wish you well'
#2524
music_df.loc[music_df['id'] == '567731.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567731.html', 'band'] = 'the brew'
music_df.loc[music_df['id'] == '567731.html', 'mics_etc'] = 'uk - 13 - _drum solo'
#2525
music_df.loc[music_df['id'] == '567732.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567732.html', 'mics_etc'] = '109 over my head'
#2527
music_df.loc[music_df['id'] == '567734.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567734.html', 'mics_etc'] = '01'
#2543
music_df.loc[music_df['id'] == '567751.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567751.html', 'mics_etc'] = '201 - mary\'s place'
#2571
music_df.loc[music_df['id'] == '567797.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567797.html', 'band'] = ''
music_df.loc[music_df['id'] == '567797.html', 'mics_etc'] = 'd1 01 my love will not let you down'
#2603
music_df.loc[music_df['id'] == '567842.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567842.html', 'mics_etc'] = '18 - ghost'
#2621
music_df.loc[music_df['id'] == '567862.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567862.html', 'mics_etc'] = '307-american land'
#2625
music_df.loc[music_df['id'] == '567867.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567867.html', 'mics_etc'] = '01'
#2628
music_df.loc[music_df['id'] == '567870.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567870.html', 'mics_etc'] = '101-highway star'
#2635
music_df.loc[music_df['id'] == '567880.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567880.html', 'mics_etc'] = '101 middle man'
#2640
music_df.loc[music_df['id'] == '567888.html', 'nodate'] = True
music_df.loc[music_df['id'] == '567888.html', 'band'] = 'sinatra'
music_df.loc[music_df['id'] == '567888.html', 'mics_etc'] = ''
#2656
music_df.loc[music_df['id'] == '126790.html', 'nodate'] = True
music_df.loc[music_df['id'] == '126790.html', 'mics_etc'] = '203 set the controls for the heart of the sun'
#2666
music_df.loc[music_df['id'] == '126803.html', 'nodate'] = True
music_df.loc[music_df['id'] == '126803.html', 'mics_etc'] = '102 look back in anger'
#2741
music_df.loc[music_df['id'] == '126904.html', 'nodate'] = True
music_df.loc[music_df['id'] == '126904.html', 'mics_etc'] = '22 25 or 6 to 4'
#2742
music_df.loc[music_df['id'] == '126905.html', 'nodate'] = True
music_df.loc[music_df['id'] == '126905.html', 'band'] = ''
music_df.loc[music_df['id'] == '126905.html', 'mics_etc'] = 'd3t03'
#2746
music_df.loc[music_df['id'] == '126912.html', 'nodate'] = True
music_df.loc[music_df['id'] == '126912.html', 'band'] = ''
music_df.loc[music_df['id'] == '126912.html', 'mics_etc'] = 'd1t01'
#2759
music_df.loc[music_df['id'] == '126935.html', 'nodate'] = True
music_df.loc[music_df['id'] == '126935.html', 'band'] = 'rolling stones'
music_df.loc[music_df['id'] == '126935.html', 'mics_etc'] = '03. - sympathy for the devil'
#2761
music_df.loc[music_df['id'] == '126940.html', 'nodate'] = True
music_df.loc[music_df['id'] == '126940.html', 'mics_etc'] = '02 banter'
#2762
music_df.loc[music_df['id'] == '126941.html', 'nodate'] = True
music_df.loc[music_df['id'] == '126941.html', 'mics_etc'] = '02 banter'
#2763
music_df.loc[music_df['id'] == '126942.html', 'nodate'] = True
music_df.loc[music_df['id'] == '126942.html', 'mics_etc'] = '02 tracks in the dust'
#2766
music_df.loc[music_df['id'] == '126946.html', 'nodate'] = True
music_df.loc[music_df['id'] == '126946.html', 'mics_etc'] = '201-roll of the dice'
#2771
music_df.loc[music_df['id'] == '126956.html', 'nodate'] = True
music_df.loc[music_df['id'] == '126956.html', 'mics_etc'] = '102 possession'
#2778
music_df.loc[music_df['id'] == '126965.html', 'nodate'] = True
music_df.loc[music_df['id'] == '126965.html', 'mics_etc'] = '103 every step of the way'
#2822
music_df.loc[music_df['id'] == '127021.html', 'nodate'] = True
music_df.loc[music_df['id'] == '127021.html', 'mics_etc'] = '201-roll of the dice'
#2823
music_df.loc[music_df['id'] == '127023.html', 'nodate'] = True
music_df.loc[music_df['id'] == '127023.html', 'band'] = 'neil young'
music_df.loc[music_df['id'] == '127023.html', 'mics_etc'] = 'd1t02. rockin\' in the free world'
#2837
music_df.loc[music_df['id'] == '127042.html', 'nodate'] = True
music_df.loc[music_df['id'] == '127042.html', 'mics_etc'] = 'd02t08 - you can look (but you better not touch)'
#2850
music_df.loc[music_df['id'] == '127059.html', 'nodate'] = True
music_df.loc[music_df['id'] == '127059.html', 'band'] = 'gregory porter'
music_df.loc[music_df['id'] == '127059.html', 'band'] = '01 - titles & introduction'
#2861
music_df.loc[music_df['id'] == '127078.html', 'nodate'] = True
music_df.loc[music_df['id'] == '127078.html', 'band'] = ''
music_df.loc[music_df['id'] == '127078.html', 'mics_etc'] = 'd3 05 monsanto years'
#2873 
music_df.loc[music_df['id'] == '127092.html', 'nodate'] = True
music_df.loc[music_df['id'] == '127092.html', 'band'] = ''
music_df.loc[music_df['id'] == '127092.html', 'mics_etc'] = 'd2t05 pay me my money down'
#2874
music_df.loc[music_df['id'] == '127094.html', 'nodate'] = True
music_df.loc[music_df['id'] == '127094.html', 'mics_etc'] = '1-10 slip away'
#2875
music_df.loc[music_df['id'] == '127095.html', 'nodate'] = True
music_df.loc[music_df['id'] == '127095.html', 'mics_etc'] = '05 - the ghost in you'
#2911
music_df.loc[music_df['id'] == '127156.html', 'nodate'] = True
music_df.loc[music_df['id'] == '127156.html', 'band'] = 'red hot chili peppers'
music_df.loc[music_df['id'] == '127156.html', 'mics_etc'] = '04 - warped'
#2913
music_df.loc[music_df['id'] == '127158.html', 'nodate'] = True
music_df.loc[music_df['id'] == '127158.html', 'mics_etc'] = '102 opening'
#2932
music_df.loc[music_df['id'] == '127178.html', 'nodate'] = True
music_df.loc[music_df['id'] == '127178.html', 'mics_etc'] = '04 - don\'t call us'
#2951
music_df.loc[music_df['id'] == '127200.html', 'nodate'] = True
music_df.loc[music_df['id'] == '127200.html', 'band'] = ''
music_df.loc[music_df['id'] == '127200.html', 'mics_etc'] = 'd1 02 heart of gold'
#2956
music_df.loc[music_df['id'] == '127205.html', 'nodate'] = True
music_df.loc[music_df['id'] == '127205.html', 'band'] = ''
music_df.loc[music_df['id'] == '127205.html', 'mics_etc'] = 'sample 2'
#2961
music_df.loc[music_df['id'] == '127212.html', 'nodate'] = True
music_df.loc[music_df['id'] == '127212.html', 'mics_etc'] = '102 opening'
#2964
music_df.loc[music_df['id'] == '127219.html', 'nodate'] = True
music_df.loc[music_df['id'] == '127219.html', 'band'] = ''
music_df.loc[music_df['id'] == '127219.html', 'mics_etc'] = 'd1 02 the promised land'

print('\nStep 11a. No band, no date: score = 0')
zeros_df = zeros_df.append(music_df.loc[(music_df['band'] == '') & (music_df['nodate'])])
music_df = music_df.loc[~music_df.index.isin(zeros_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

print('\nStep 11b. Band (even if incorrect), no datet: score = 1')
ones_df = ones_df.append(music_df.loc[(music_df['band'] != '') & (music_df['nodate'])])
music_df = music_df.loc[~music_df.index.isin(ones_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

In [ ]:
print('Step 12. The preceding missed some rows and I don\'t know why, the hell with it, set them by hand.')
#2353
music_df.loc[music_df['id'] == '567513.html', 'parsed_date'] = pandas.to_datetime('2016-07-22')
music_df.loc[music_df['id'] == '567513.html', 'band'] = 'senior service'
music_df.loc[music_df['id'] == '567513.html', 'mics_etc'] = 'chatham_track_12'
threes_df = threes_df.append(music_df.loc[music_df['id'] == '567513.html'])

#2397
music_df.loc[music_df['id'] == '567572.html', 'parsed_date'] = pandas.to_datetime('2016-07-22')
music_df.loc[music_df['id'] == '567572.html', 'band'] = 'senior service'
music_df.loc[music_df['id'] == '567572.html', 'mics_etc'] = '3_chatham_track_18'
threes_df = threes_df.append(music_df.loc[music_df['id'] == '567572.html'])

#2459
music_df.loc[music_df['id'] == '567649.html', 'parsed_date'] = pandas.to_datetime('2016-07-23')
music_df.loc[music_df['id'] == '567649.html', 'band'] = 'claim'
music_df.loc[music_df['id'] == '567649.html', 'mics_etc'] = 'rochester_track_20'
threes_df = threes_df.append(music_df.loc[music_df['id'] == '567649.html'])

#2468
music_df.loc[music_df['id'] == '567662.html', 'parsed_date'] = pandas.to_datetime('2016-07-23')
music_df.loc[music_df['id'] == '567662.html', 'band'] = 'bob collins + full nelson'
music_df.loc[music_df['id'] == '567662.html', 'mics_etc'] = 'rochesterr_track_04'
threes_df = threes_df.append(music_df.loc[music_df['id'] == '567662.html'])

#2542
music_df.loc[music_df['id'] == '567750.html', 'parsed_date'] = pandas.to_datetime('2016-07-23')
music_df.loc[music_df['id'] == '567750.html', 'band'] = 'peter astor'
music_df.loc[music_df['id'] == '567750.html', 'mics_etc'] = 'rochesterr_track_07'
threes_df = threes_df.append(music_df.loc[music_df['id'] == '567750.html'])


'''
These rows are OK now because REASONS!
#1612
music_df.loc[music_df['id'] == '566546.html', 'parsed_date'] = pandas.to_datetime('1978-04-01')
music_df.loc[music_df['id'] == '566546.html', 'band'] = 'jam'
music_df.loc[music_df['id'] == '566546.html', 'mics_etc'] = '4acres08'
threes_df = threes_df.append(music_df.loc[music_df['id'] == '566546.html'])

#1864
music_df.loc[music_df['id'] == '566890.html', 'parsed_date'] = pandas.to_datetime('1978-04-01')
music_df.loc[music_df['id'] == '566890.html', 'band'] = 'jam'
music_df.loc[music_df['id'] == '566890.html', 'mics_etc'] = 'paris_mc01'
twos_df = twos_df.append(music_df.loc[music_df['id'] == '566890.html'])

#1979
music_df.loc[music_df['id'] == '567038.html', 'parsed_date'] = pandas.to_datetime('2009-09-22')
music_df.loc[music_df['id'] == '567038.html', 'band'] = ''
music_df.loc[music_df['id'] == '567038.html', 'mics_etc'] = '01 essen main part'
ones_df = ones_df.append(music_df.loc[music_df['id'] == '567038.html'])

#2000
music_df.loc[music_df['id'] == '567064.html', 'parsed_date'] = pandas.to_datetime('2003-09-23')
music_df.loc[music_df['id'] == '567064.html', 'band'] = 'radiohead'
music_df.loc[music_df['id'] == '567064.html', 'abbrev_type'] = 'notfound'
music_df.loc[music_df['id'] == '567064.html', 'mics_etc'] = 'shoreline_25 everything in its right place.'
threes_df = threes_df.append(music_df.loc[music_df['id'] == '567064.html'])


#2348
music_df.loc[music_df['id'] == '567507.html', 'parsed_date'] = pandas.to_datetime('2016-07-26')
music_df.loc[music_df['id'] == '567507.html', 'band'] = 'boris'
music_df.loc[music_df['id'] == '567507.html', 'abbrev_type'] = 'notfound'
music_df.loc[music_df['id'] == '567507.html', 'mics_etc'] = '00'
threes_df = threes_df.append(music_df.loc[music_df['id'] == '567507.html'])
'''

music_df = music_df.loc[~music_df.index.isin(ones_df.index)]
music_df = music_df.loc[~music_df.index.isin(twos_df.index)]
music_df = music_df.loc[~music_df.index.isin(threes_df.index)]
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

In [ ]:
# should add up to 2973
len(zeros_df) #1299
len(ones_df) #179
len(twos_df) #421
len(threes_df) #428
len(fours_df) #935
len(zeros_df) + len(ones_df) + len(twos_df) + len(threes_df) + len(fours_df) #2973
#YAY!

zeros_df_bk = zeros_df
ones_df_bk = ones_df
twos_df_bk = twos_df
threes_df_bk = threes_df
fours_df_bk = fours_df
print('ok')

In [ ]:
zeros_df = zeros_df_bk
ones_df = ones_df_bk
twos_df = twos_df_bk
threes_df = threes_df_bk
fours_df = fours_df_bk


outmusic_df = pandas.DataFrame(data=None, columns=['filename', 'filetype', 'band', 'abbrev_type', 'date', 'date_format', 'mics_etc', 'score'])

fours_df['score'] = 4
fours_df['date'] = fours_df['full_date']
fours_df['mics_etc'] = fours_df['full_date_mics_etc']

outmusic_df = outmusic_df.append(fours_df[['filename', 'filetype', 'band', 'abbrev_type', 'date', 'date_format', 'mics_etc', 'score']])


threes_df['score'] = 3
threes_df['date'] = pandas.to_datetime(np.nan)
threes_df.loc[threes_df['full_date'].notnull(), 'date'] = threes_df['full_date']
threes_df.loc[threes_df['yy_date'].notnull(), 'date'] = threes_df['yy_date']
threes_df.loc[threes_df['parsed_date'].notnull(), 'date'] = threes_df['parsed_date']
threes_df['mics_etc'] = ''
threes_df.loc[threes_df['full_date_mics_etc'].notnull(), 'mics_etc'] = threes_df['full_date_mics_etc']
threes_df.loc[threes_df['yy_date_mics_etc'].notnull(), 'mics_etc'] = threes_df['yy_date_mics_etc']
threes_df.loc[threes_df['parsed_date_mics_etc2'].notnull(), 'mics_etc'] = threes_df['parsed_date_mics_etc2']
outmusic_df = outmusic_df.append(threes_df[['filename', 'filetype', 'band', 'abbrev_type', 'date', 'date_format', 'mics_etc', 'score']])

twos_df['score'] = 2
twos_df['date'] = pandas.to_datetime(np.nan)
twos_df.loc[twos_df['full_date'].notnull(), 'date'] = twos_df['full_date']
twos_df.loc[twos_df['yy_date'].notnull(), 'date'] = twos_df['yy_date']
twos_df.loc[twos_df['parsed_date'].notnull(), 'date'] = twos_df['parsed_date']
twos_df['mics_etc'] = ''
twos_df.loc[twos_df['full_date_mics_etc'].notnull(), 'mics_etc'] = twos_df['full_date_mics_etc']
twos_df.loc[twos_df['yy_date_mics_etc'].notnull(), 'mics_etc'] = twos_df['yy_date_mics_etc']
twos_df.loc[twos_df['parsed_date_mics_etc2'].notnull(), 'mics_etc'] = twos_df['parsed_date_mics_etc2']
outmusic_df = outmusic_df.append(twos_df[['filename', 'filetype', 'band', 'abbrev_type', 'date', 'date_format', 'mics_etc', 'score']])

ones_df['score'] = 2
ones_df['date'] = pandas.to_datetime(np.nan)
ones_df.loc[ones_df['full_date'].notnull(), 'date'] = ones_df['full_date']
ones_df.loc[ones_df['yy_date'].notnull(), 'date'] = ones_df['yy_date']
ones_df.loc[ones_df['parsed_date'].notnull(), 'date'] = ones_df['parsed_date']
ones_df['mics_etc'] = ''
ones_df.loc[ones_df['full_date_mics_etc'].notnull(), 'mics_etc'] = ones_df['full_date_mics_etc']
ones_df.loc[ones_df['yy_date_mics_etc'].notnull(), 'mics_etc'] = ones_df['yy_date_mics_etc']
ones_df.loc[ones_df['parsed_date_mics_etc2'].notnull(), 'mics_etc'] = ones_df['parsed_date_mics_etc2']
outmusic_df = outmusic_df.append(ones_df[['filename', 'filetype', 'band', 'abbrev_type', 'date', 'date_format', 'mics_etc', 'score']])

zeros_df['score'] = 0
zeros_df['date'] = pandas.to_datetime(np.nan)
zeros_df['band'] = ''
zeros_df['mics_etc'] = ''
outmusic_df = outmusic_df.append(zeros_df[['filename', 'filetype', 'band', 'abbrev_type', 'date', 'date_format', 'mics_etc', 'score']])

outmusic_df.index.name = 'rownumber'
print('Step 13: combine all data into output dataset.\n')
print('{:,.0f} files in original input dataset.'.format(len(inmusic_df)))
print('{:,.0f} files in output dataset.'.format(len(outmusic_df)))

In [ ]:
print('Step 14: remove that .cue file per Scott on 2018-02-08\n')
outmusic_df = outmusic_df[~outmusic_df['filename'].apply(lambda x: '.cue' in x)]
print('{:,.0f} files remain in output dataset.'.format(len(outmusic_df)))

In [ ]:
outmusic_df = outmusic_df.sort_values(by='date_format').sort_values(by='abbrev_type').sort_values(by='score', ascending=False)
outmusic_df.to_csv('outmusic4.csv')
#print('ok')
print(len(outmusic_df))

In [ ]:

# REMOVE TRAILING CHARACTERS FROM BAND NAME
'''
print('/'+music_df['band'].loc[2762]+'/')

# Parse away markers (e.g. _) off beginning and end of band name
music_df.loc[
    music_df['band'].apply(
        lambda x: re.search(endmarkers, x) != None)
, 'band'] = music_df['band'][
    music_df['band'].apply(
        lambda x: re.search(endmarkers, x) != None)
].apply(
    lambda x: 
    x[0:re.search(endmarkers, x).start()].lower().strip()
)
'''
